In [362]:
import numpy as np
import pandas as pd
import os
import glob
import sys
sys.path.append('../scripts')

import api
import shutil

import os
from utils import cleanCmpnyNm, getGrpCmpnyDict, convertCmpnyNm, convertCmpnyNm
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from time import sleep
import re

In [5]:
dfTrans19 = pd.read_excel('../data/transactions-mrg/2019/stacked.xlsx')
dfTrans19.head()

,매출회사,매입회사,매출액,국내계열사계(매출액),해외계열사계(매출액),국내매출액,해외매출액,매출액총계,year2,year,grpname2,매출사상/비,매출사금융/비금융,매출액Val,매입사상/비,매입사금융/비금융,grpname,grpcode,매출회사id,매입회사id
0,삼성전자,삼성물산,78676.0,2269950.0,149436938.0,16821275.0,153560595.0,170381870.0,2019,2018,삼성,상장,비금융,170381870.0,상장,비금융,samsung,511,380725,810053.0
1,삼성전자,삼성바이오에피스,1.0,2269950.0,149436938.0,16821275.0,153560595.0,170381870.0,2019,2018,삼성,상장,비금융,170381870.0,비상장,비금융,samsung,511,380725,18082125.0
2,삼성전자,삼성디스플레이,164404.0,2269950.0,149436938.0,16821275.0,153560595.0,170381870.0,2019,2018,삼성,상장,비금융,170381870.0,비상장,비금융,samsung,511,380725,18082126.0
3,삼성전자,세메스,5.0,2269950.0,149436938.0,16821275.0,153560595.0,170381870.0,2019,2018,삼성,상장,비금융,170381870.0,비상장,비금융,samsung,511,380725,314528.0
4,삼성전자,삼성전자서비스,456109.0,2269950.0,149436938.0,16821275.0,153560595.0,170381870.0,2019,2018,삼성,상장,비금융,170381870.0,비상장,비금융,samsung,511,380725,952133.0


In [6]:
dfTrans19[dfTrans19['매출회사id'].isna()]

,매출회사,매입회사,매출액,국내계열사계(매출액),해외계열사계(매출액),국내매출액,해외매출액,매출액총계,year2,year,grpname2,매출사상/비,매출사금융/비금융,매출액Val,매입사상/비,매입사금융/비금융,grpname,grpcode,매출회사id,매입회사id


In [7]:
print(list(dfTrans19[dfTrans19['매입회사id'].isna()]['매입회사'].unique()))

['SKChinaCompany,Ltd', 'SKC&CBeijingCo.,Ltd', 'ESSENCORELimited', 'SKcomputerandcommunicationL.L.C', 'SKMaterials(Jiangsu)Co.,Ltd', 'SKBiotekIrelandLimited', 'SKBiotekUSA,Inc', 'SKUSA,Inc', 'SKE&PCompany', 'SKShanghaiAsphaltCo.,Ltd', 'SKGlobalChemicalInternationalTrading(Shanghai)Co.,Ltd', 'SKGlobalChemical(China)HoldingCo.,Ltd', 'SKGlobalChemicalSingaporePte.Ltd', 'NingboSKPerformanceRubberCo.,Ltd', 'SKGlobalChemicalJapanCo.,Ltd', 'SKPrimacorEurope,S.L.U', 'SKPrimacorAmericasLLC', 'SKSaranAmericasLLC', 'SKGCAmericas,Inc', 'SKGlobalChemicalInternationalTrading(Guangzhou)Co.,Ltd', 'SKEnergyLubricants(Tianjin)Co.,Ltd', 'PT.PatraSK', 'SKLubricantsJapanCo.,Ltd', 'SKLubricantsEuropeB.V', 'SKLubricantsAmericas,Inc', 'SKEnergyInternationalPte,Ltd', 'SKEnergyEurope,Ltd', 'SKEnergyAmericasInc', 'FSKL&S(Shanghai)Co.,Ltd', 'SKhynixSemiconductor(China)Ltd.(SKHYCL)', 'HITECHSemiconductor(Wuxi)Co.,Ltd.(HITECH)', 'SKhynixSemiconductor(Chongqing)Ltd.(SKHYCQL)', 'SKCEuropeGmbH', 'SKC,Inc', 'SKC(Jiangsu

In [24]:
dfTrans19[dfTrans19['매입회사'] == '동삼현대한매고분']

,매출회사,매입회사,매출액,국내계열사계(매출액),해외계열사계(매출액),국내매출액,해외매출액,매출액총계,year2,year,grpname2,매출사상/비,매출사금융/비금융,매출액Val,매입사상/비,매입사금융/비금융,grpname,grpcode,매출회사id,매입회사id
8477,이에이치케이이앤에스,동삼현대한매고분,67.0,0.0,67.0,0.0,67.0,67.0,2019,2018,현대백화점,비상장,비금융,67.0,NaN,NaN,ehyundai,38035,2101603529,NaN


In [25]:
dfTrans19.loc[8477, '매입회사id'] = 2830000000

In [26]:
dfTrans19[dfTrans19['매입회사'] == '드림라인쏠라파워1호']

,매출회사,매입회사,매출액,국내계열사계(매출액),해외계열사계(매출액),국내매출액,해외매출액,매출액총계,year2,year,grpname2,매출사상/비,매출사금융/비금융,매출액Val,매입사상/비,매입사금융/비금융,grpname,grpcode,매출회사id,매입회사id
8883,드림라인,드림라인쏠라파워1호,2.0,763.0,NaN,4984.0,365.0,5349.0,2019,2018,한국투자금융,비상장,비금융,169316.0,NaN,NaN,kinv,2702504,818160,NaN
8884,드림라인,드림라인쏠라파워1호,9.0,763.0,NaN,4984.0,365.0,5349.0,2019,2018,한국투자금융,비상장,비금융,169316.0,NaN,NaN,kinv,2702504,818160,NaN
8888,드림마크원,드림라인쏠라파워1호,19.0,4019.0,NaN,7.0,20905.0,20912.0,2019,2018,한국투자금융,비상장,비금융,7120.0,NaN,NaN,kinv,2702504,1902101842,NaN


In [27]:
dfTrans19.loc[8883, '매입회사id'] = 1902403697
dfTrans19.loc[8884, '매입회사id'] = 1902403697
dfTrans19.loc[8888, '매입회사id'] = 1902403697

In [28]:
dfTrans19[dfTrans19['매입회사'] == '탱그램디자인연']

,매출회사,매입회사,매출액,국내계열사계(매출액),해외계열사계(매출액),국내매출액,해외매출액,매출액총계,year2,year,grpname2,매출사상/비,매출사금융/비금융,매출액Val,매입사상/비,매입사금융/비금융,grpname,grpcode,매출회사id,매입회사id
9894,카카오인베스트먼트,탱그램디자인연,5.0,28.0,NaN,28.0,NaN,NaN,2019,2018,카카오,비상장,비금융,28.0,NaN,NaN,kakao,36027,1902702361,NaN


In [29]:
dfTrans19.loc[9894, '매입회사id'] = 16028567

In [30]:
dfTrans19[dfTrans19['매입회사'] == '디비라이텍']

,매출회사,매입회사,매출액,국내계열사계(매출액),해외계열사계(매출액),국내매출액,해외매출액,매출액총계,year2,year,grpname2,매출사상/비,매출사금융/비금융,매출액Val,매입사상/비,매입사금융/비금융,grpname,grpcode,매출회사id,매입회사id
10890,디비아이엔씨,디비라이텍,285.0,46302.0,0.0,122170.0,84669.0,206839.0,2019,2018,DB,상장,비금융,206839.0,NaN,NaN,DB,365,352098,NaN
10913,디비메탈,디비라이텍,0.0,56028.0,94048.0,187555.0,324816.0,512371.0,2019,2018,DB,비상장,비금융,512372.0,NaN,NaN,DB,365,12097776,NaN
10936,디비월드,디비라이텍,0.0,1101.0,0.0,12140.0,0.0,12140.0,2019,2018,DB,비상장,비금융,12140.0,NaN,NaN,DB,365,910813,NaN
10959,디비하이텍,디비라이텍,0.0,0.0,0.0,134995.0,534284.0,669279.0,2019,2018,DB,상장,비금융,669279.0,NaN,NaN,DB,365,350354,NaN
10982,동부철,디비라이텍,0.0,0.0,0.0,0.0,0.0,0.0,2019,2018,DB,비상장,비금융,0.0,NaN,NaN,DB,365,65122,NaN
11005,디비스탁인베스트,디비라이텍,0.0,0.0,0.0,256.0,0.0,256.0,2019,2018,DB,비상장,비금융,256.0,NaN,NaN,DB,365,19071407,NaN
11028,디비에프아이에스,디비라이텍,0.0,47260.0,0.0,47269.0,0.0,47269.0,2019,2018,DB,비상장,비금융,47269.0,NaN,NaN,DB,365,24038019,NaN
11051,디비인베스트,디비라이텍,0.0,0.0,0.0,933.0,0.0,933.0,2019,2018,DB,비상장,비금융,933.0,NaN,NaN,DB,365,16003715,NaN
11074,디비저축은행,디비라이텍,0.0,79.0,0.0,75424.0,0.0,75424.0,2019,2018,DB,비상장,금융,75423.0,NaN,NaN,DB,365,881562,NaN
11097,디비금융서비스,디비라이텍,0.0,3675.0,0.0,13311.0,0.0,13311.0,2019,2018,DB,비상장,금융,13311.0,NaN,NaN,DB,365,23044292,NaN


In [31]:
dfTrans19.loc[10890, '매입회사id'] = 137656
dfTrans19.loc[10913, '매입회사id'] = 137656
dfTrans19.loc[10936, '매입회사id'] = 137656
dfTrans19.loc[10959, '매입회사id'] = 137656
dfTrans19.loc[10982, '매입회사id'] = 137656
dfTrans19.loc[11005, '매입회사id'] = 137656
dfTrans19.loc[11028, '매입회사id'] = 137656
dfTrans19.loc[11051, '매입회사id'] = 137656
dfTrans19.loc[11074, '매입회사id'] = 137656
dfTrans19.loc[11097, '매입회사id'] = 137656
dfTrans19.loc[11120, '매입회사id'] = 137656
dfTrans19.loc[11143, '매입회사id'] = 137656
dfTrans19.loc[11166, '매입회사id'] = 137656
dfTrans19.loc[11189, '매입회사id'] = 137656
dfTrans19.loc[11212, '매입회사id'] = 137656
dfTrans19.loc[11235, '매입회사id'] = 137656
dfTrans19.loc[11258, '매입회사id'] = 137656
dfTrans19.loc[11281, '매입회사id'] = 137656
dfTrans19.loc[11304, '매입회사id'] = 137656
dfTrans19.loc[11327, '매입회사id'] = 137656

In [32]:
dfTrans19[dfTrans19['매입회사'] == '유진스타제2호사모투자']

,매출회사,매입회사,매출액,국내계열사계(매출액),해외계열사계(매출액),국내매출액,해외매출액,매출액총계,year2,year,grpname2,매출사상/비,매출사금융/비금융,매출액Val,매입사상/비,매입사금융/비금융,grpname,grpcode,매출회사id,매입회사id
12233,유진프라이빗에쿼티,유진스타제2호사모투자,513.0,3007.0,NaN,3282.0,NaN,NaN,2019,2018,유진,비상장,비금융,3282.0,NaN,NaN,eugenes,653,1302904046,NaN


In [33]:
dfTrans19.loc[12233, '매입회사id'] = 1803200394

In [8]:
dfTrans19[dfTrans19['매입회사'] == '디디아이에스와이230위탁관리부동산투자회사']

,매출회사,매입회사,매출액,국내계열사계(매출액),해외계열사계(매출액),국내매출액,해외매출액,매출액총계,year2,year,grpname2,매출사상/비,매출사금융/비금융,매출액Val,매입사상/비,매입사금융/비금융,grpname,grpcode,매출회사id,매입회사id
1664,디앤디인베스트먼트,디디아이에스와이230위탁관리부동산투자회사,1620.0,1620.0,NaN,1720.0,NaN,NaN,2019,2018,에스케이,비상장,비금융,1720.0,NaN,NaN,sk,535,1903700469,NaN


In [9]:
dfTrans19.loc[1664, '매입회사id'] = 2101809024

In [11]:
dfTrans19[dfTrans19['매입회사'] == '드림라인솔라파워5호']

,매출회사,매입회사,매출액,국내계열사계(매출액),해외계열사계(매출액),국내매출액,해외매출액,매출액총계,year2,year,grpname2,매출사상/비,매출사금융/비금융,매출액Val,매입사상/비,매입사금융/비금융,grpname,grpcode,매출회사id,매입회사id
5611,한화갤러리아타임월드,드림라인솔라파워5호,0.0,3446.0,0.0,332681.0,0.0,332681.0,2019,2018,한화,상장,비금융,332681.0,NaN,NaN,hanwha,913,620580,NaN
5753,한화에스테이트,드림라인솔라파워5호,0.0,86755.0,0.0,99950.0,NaN,NaN,2019,2018,한화,비상장,비금융,99950.0,NaN,NaN,hanwha,913,39667,NaN
5940,한화시스템,드림라인솔라파워5호,0.0,230785.0,4593.0,1095012.0,26404.0,1121416.0,2019,2018,한화,비상장,비금융,1121416.0,NaN,NaN,hanwha,913,128888,NaN


In [12]:
dfTrans19.loc[5611, '매입회사id'] = 1902902067
dfTrans19.loc[5753, '매입회사id'] = 386383
dfTrans19.loc[5940, '매입회사id'] = 386383

In [13]:
dfTrans19[dfTrans19['매입회사'] == '센트럴관광개발']

,매출회사,매입회사,매출액,국내계열사계(매출액),해외계열사계(매출액),국내매출액,해외매출액,매출액총계,year2,year,grpname2,매출사상/비,매출사금융/비금융,매출액Val,매입사상/비,매입사금융/비금융,grpname,grpcode,매출회사id,매입회사id
6632,신세계,센트럴관광개발,388.0,196180.0,NaN,1739327.0,NaN,NaN,2019,2018,신세계,상장,비금융,1739327.0,NaN,NaN,shinsegae,577,620084,NaN
6693,신세계센트럴시티,센트럴관광개발,2339.0,101977.0,NaN,142969.0,NaN,NaN,2019,2018,신세계,비상장,비금융,172538.0,NaN,NaN,shinsegae,577,711268,NaN
6718,까사미아,센트럴관광개발,2472.0,7115.0,NaN,109567.0,NaN,NaN,2019,2018,신세계,비상장,비금융,109567.0,NaN,NaN,shinsegae,577,274224,NaN
6778,신세계조선호텔,센트럴관광개발,93.0,14890.0,NaN,191684.0,NaN,NaN,2019,2018,신세계,비상장,비금융,191684.0,NaN,NaN,shinsegae,577,630306,NaN
6807,신세계푸드,센트럴관광개발,187.0,397221.0,NaN,1263417.0,268.0,1263685.0,2019,2018,신세계,상장,비금융,1263685.0,NaN,NaN,shinsegae,577,40807,NaN
6837,신세계엘앤비,센트럴관광개발,32.0,56681.0,NaN,93560.0,NaN,NaN,2019,2018,신세계,비상장,비금융,93560.0,NaN,NaN,shinsegae,577,13072689,NaN
6861,신세계건설,센트럴관광개발,94209.0,675215.0,229.0,1084040.0,229.0,1084269.0,2019,2018,신세계,상장,비금융,1084269.0,NaN,NaN,shinsegae,577,100960,NaN
6897,신세계영랑호리조트,센트럴관광개발,0.0,1581.0,NaN,8171.0,NaN,NaN,2019,2018,신세계,비상장,비금융,8171.0,NaN,NaN,shinsegae,577,494258,NaN
6951,신세계아이앤씨,센트럴관광개발,6547.0,286625.0,NaN,373529.0,2.0,373531.0,2019,2018,신세계,상장,비금융,373531.0,NaN,NaN,shinsegae,577,538787,NaN


In [15]:
dfTrans19.loc[6632, '매입회사id'] = 73540
dfTrans19.loc[6693, '매입회사id'] = 73540
dfTrans19.loc[6718, '매입회사id'] = 73540
dfTrans19.loc[6778, '매입회사id'] = 73540
dfTrans19.loc[6807, '매입회사id'] = 73540
dfTrans19.loc[6837, '매입회사id'] = 73540
dfTrans19.loc[6861, '매입회사id'] = 73540
dfTrans19.loc[6897, '매입회사id'] = 73540
dfTrans19.loc[6951, '매입회사id'] = 73540

In [16]:
dfTrans19[dfTrans19['매입회사'] == '케이티씨엔피그로쓰챔프2011의2호사모투자']

,매출회사,매입회사,매출액,국내계열사계(매출액),해외계열사계(매출액),국내매출액,해외매출액,매출액총계,year2,year,grpname2,매출사상/비,매출사금융/비금융,매출액Val,매입사상/비,매입사금융/비금융,grpname,grpcode,매출회사id,매입회사id
7908,네오플럭스,케이티씨엔피그로쓰챔프2011의2호사모투자,8425.0,10350.0,NaN,22219.0,NaN,NaN,2019,2018,두산,비상장,금융,22219.0,NaN,NaN,doosan,391,1903209021,NaN


In [17]:
dfTrans19.loc[7908, '매입회사id'] = 19066564

In [18]:
dfTrans19[dfTrans19['매입회사'] == 'PCA생명보험']

,매출회사,매입회사,매출액,국내계열사계(매출액),해외계열사계(매출액),국내매출액,해외매출액,매출액총계,year2,year,grpname2,매출사상/비,매출사금융/비금융,매출액Val,매입사상/비,매입사금융/비금융,grpname,grpcode,매출회사id,매입회사id
8335,미래에셋대우,PCA생명보험,231.0,18568.0,92.0,8999289.0,2278029.0,11277318.0,2019,2018,미래에셋,상장,금융,11277318.0,NaN,NaN,miraeasset,2602105,860085,NaN
8365,미래에셋자산운용,PCA생명보험,46.0,24066.0,179.0,265349.0,9176.0,274525.0,2019,2018,미래에셋,비상장,금융,274525.0,NaN,NaN,miraeasset,2602105,490050,NaN
8392,미래에셋컨설팅,PCA생명보험,10.0,4243.0,NaN,108786.0,NaN,NaN,2019,2018,미래에셋,비상장,비금융,108786.0,NaN,NaN,miraeasset,2602105,13045914,NaN


In [20]:
dfTrans19.loc[8335, '매입회사id'] = 821683
dfTrans19.loc[8365, '매입회사id'] = 821683
dfTrans19.loc[8392, '매입회사id'] = 821683

In [22]:
dfTrans19[dfTrans19['매입회사'] == '미래에셋파트너스5호']

,매출회사,매입회사,매출액,국내계열사계(매출액),해외계열사계(매출액),국내매출액,해외매출액,매출액총계,year2,year,grpname2,매출사상/비,매출사금융/비금융,매출액Val,매입사상/비,매입사금융/비금융,grpname,grpcode,매출회사id,매입회사id
8339,미래에셋대우,미래에셋파트너스5호,36.0,18568.0,92.0,8999289.0,2278029.0,11277318.0,2019,2018,미래에셋,상장,금융,11277318.0,NaN,NaN,miraeasset,2602105,860085,NaN


In [23]:
dfTrans19.loc[8339, '매입회사id'] = 13098472

In [34]:
idx = dfTrans19[dfTrans19['매입회사id'].isna()].index

In [35]:
dfTrans19.loc[idx, '매입회사id'] = 5350000000

In [38]:
dfUltown = pd.read_stata("../data/almeida/2019/out/almeida_own1_2019.dta")
dfUltown.head()

,year,year2,grpname2,grpname,grpcode,firmname,firmid,fid,ultown,avpos,sd,steps,loop,lastcontrol,central
0,2018.0,2019.0,애경,aekyung,615,우영운수,47919,1.0,1.000000,1.000000,1.0,NaN,0.0,100.0,0.000000
1,2018.0,2019.0,애경,aekyung,615,수원애경역사,73220,2.0,0.605655,3.206202,2.0,NaN,0.0,64.0,0.000000
2,2018.0,2019.0,애경,aekyung,615,애드미션,95760,3.0,0.852500,1.000000,1.0,NaN,0.0,85.0,0.000000
3,2018.0,2019.0,애경,aekyung,615,에이케이레저,99486,4.0,1.000000,2.569844,2.0,NaN,0.0,100.0,0.000000
4,2018.0,2019.0,애경,aekyung,615,에이케이에스앤디,125359,5.0,0.719310,2.206202,1.0,NaN,0.0,64.0,6.736839


In [39]:
dfUltown['매출회사 실질지분율'] = dfUltown['ultown']
dfUltown['매입회사 실질지분율'] = dfUltown['ultown']
dfUltown['매출회사id'] = dfUltown['firmid']
dfUltown['매입회사id'] = dfUltown['firmid']

In [41]:
dfUltown['매출회사id'].dtype

dtype('int32')

In [42]:
dfTrans19['매출회사id'].dtype

dtype('int64')

In [43]:
dfTrans19['매입회사id'].dtype

dtype('float64')

In [44]:
dfUltown['매입회사id'].dtype

dtype('int32')

In [45]:
dfTrans19['매입회사id'] = dfTrans19['매입회사id'].astype('int32')

In [46]:
dfTrans19Vr = dfTrans19.merge(dfUltown[['매출회사id', '매출회사 실질지분율']], on='매출회사id', how='left')\
                       .merge(dfUltown[['매입회사id', '매입회사 실질지분율']], on='매입회사id', how='left')
dfTrans19Vr.head()

,매출회사,매입회사,매출액,국내계열사계(매출액),해외계열사계(매출액),국내매출액,해외매출액,매출액총계,year2,year,...,매출사금융/비금융,매출액Val,매입사상/비,매입사금융/비금융,grpname,grpcode,매출회사id,매입회사id,매출회사 실질지분율,매입회사 실질지분율
0,삼성전자,삼성물산,78676.0,2269950.0,149436938.0,16821275.0,153560595.0,170381870.0,2019,2018,...,비금융,170381870.0,상장,비금융,samsung,511,380725,810053,0.08541,0.311626
1,삼성전자,삼성바이오에피스,1.0,2269950.0,149436938.0,16821275.0,153560595.0,170381870.0,2019,2018,...,비금융,170381870.0,비상장,비금융,samsung,511,380725,18082125,0.08541,0.081307
2,삼성전자,삼성디스플레이,164404.0,2269950.0,149436938.0,16821275.0,153560595.0,170381870.0,2019,2018,...,비금융,170381870.0,비상장,비금융,samsung,511,380725,18082126,0.08541,0.074901
3,삼성전자,세메스,5.0,2269950.0,149436938.0,16821275.0,153560595.0,170381870.0,2019,2018,...,비금융,170381870.0,비상장,비금융,samsung,511,380725,314528,0.08541,0.078187
4,삼성전자,삼성전자서비스,456109.0,2269950.0,149436938.0,16821275.0,153560595.0,170381870.0,2019,2018,...,비금융,170381870.0,비상장,비금융,samsung,511,380725,952133,0.08541,0.084836


In [48]:
dfTrans19.to_excel('../data/transactions-report/transactions2019.xlsx')

In [4]:
dfTrans19 = pd.read_excel('../data/transactions-report/transactions2019.xlsx')
dfTrans19.head()

,매출회사,매입회사,매출액,국내계열사계(매출액),해외계열사계(매출액),국내매출액,해외매출액,매출액총계,year2,year,grpname2,매출사상/비,매출사금융/비금융,매출액Val,매입사상/비,매입사금융/비금융,grpname,grpcode,매출회사id,매입회사id
0,삼성전자,삼성물산,78676.0,2269950.0,149436938.0,16821275.0,153560595.0,170381870.0,2019,2018,삼성,상장,비금융,170381870.0,상장,비금융,samsung,511,380725,810053
1,삼성전자,삼성바이오에피스,1.0,2269950.0,149436938.0,16821275.0,153560595.0,170381870.0,2019,2018,삼성,상장,비금융,170381870.0,비상장,비금융,samsung,511,380725,18082125
2,삼성전자,삼성디스플레이,164404.0,2269950.0,149436938.0,16821275.0,153560595.0,170381870.0,2019,2018,삼성,상장,비금융,170381870.0,비상장,비금융,samsung,511,380725,18082126
3,삼성전자,세메스,5.0,2269950.0,149436938.0,16821275.0,153560595.0,170381870.0,2019,2018,삼성,상장,비금융,170381870.0,비상장,비금융,samsung,511,380725,314528
4,삼성전자,삼성전자서비스,456109.0,2269950.0,149436938.0,16821275.0,153560595.0,170381870.0,2019,2018,삼성,상장,비금융,170381870.0,비상장,비금융,samsung,511,380725,952133


In [28]:
dfTrans20 = pd.read_excel('../data/transactions-report/transactions2020.xlsx')
dfTrans20.head()

,매출회사,매입회사,매출액,국내계열사계(매출액),해외계열사계(매출액),국내매출액,해외매출액,매출액총계,year2,year,grpname2,매출사상/비,매출사금융/비금융,매출액Val,매입사상/비,매입사금융/비금융,grpname,grpcode,매출회사id,매입회사id
0,삼성전자,삼성물산,110238.0,2422149.0,130563681.0,20300939.0,134471920.0,154772859.0,2020,2019,삼성,상장,비금융,154772859.0,상장,비금융,samsung,511,380725,810053
1,삼성전자,삼성바이오에피스,4.0,2422149.0,130563681.0,20300939.0,134471920.0,154772859.0,2020,2019,삼성,상장,비금융,154772859.0,비상장,비금융,samsung,511,380725,18082125
2,삼성전자,삼성디스플레이,279363.0,2422149.0,130563681.0,20300939.0,134471920.0,154772859.0,2020,2019,삼성,상장,비금융,154772859.0,비상장,비금융,samsung,511,380725,18082126
3,삼성전자,세메스,7.0,2422149.0,130563681.0,20300939.0,134471920.0,154772859.0,2020,2019,삼성,상장,비금융,154772859.0,비상장,비금융,samsung,511,380725,314528
4,삼성전자,삼성전자서비스,453084.0,2422149.0,130563681.0,20300939.0,134471920.0,154772859.0,2020,2019,삼성,상장,비금융,154772859.0,비상장,비금융,samsung,511,380725,952133


In [30]:
dfTrans21 = pd.read_excel('../data/transactions-report/transactions2021.xlsx')
dfTrans21.head()

,매출회사,매입회사,매출액,국내계열사계(매출액),해외계열사계(매출액),국내매출액,해외매출액,매출액총계,year2,year,...,매출사금융/비금융,매출액Val,매입사상/비,매입사금융/비금융,grpname,grpcode,매출회사id,매입회사id,매출회사 실질지분율,매입회사 실질지분율
0,삼성전자,삼성물산,86705.0,3142513.0,141902290.0,19833135.0,146478056.0,166311191.0,2021,2020,...,비금융,1.660000e+14,상장,비금융,samsung,511,380725,810053,0.085685,0.316255
1,삼성전자,삼성바이오에피스,0.0,3142513.0,141902290.0,19833135.0,146478056.0,166311191.0,2021,2020,...,비금융,1.660000e+14,비상장,비금융,samsung,511,380725,18082125,0.085685,0.082313
2,삼성전자,삼성디스플레이,196823.0,3142513.0,141902290.0,19833135.0,146478056.0,166311191.0,2021,2020,...,비금융,1.660000e+14,비상장,비금융,samsung,511,380725,18082126,0.085685,0.075274
3,삼성전자,삼성전자서비스,465778.0,3142513.0,141902290.0,19833135.0,146478056.0,166311191.0,2021,2020,...,비금융,1.660000e+14,비상장,비금융,samsung,511,380725,952133,0.085685,0.085109
4,삼성전자,삼성전자판매,2225711.0,3142513.0,141902290.0,19833135.0,146478056.0,166311191.0,2021,2020,...,비금융,1.660000e+14,비상장,비금융,samsung,511,380725,102357,0.085685,0.085685


# 전체 재벌

In [6]:
len(dfTrans19)

12797

In [14]:
dfTrans19[dfTrans19['매입회사id'] == -2147483648]

,매출회사,매입회사,매출액,국내계열사계(매출액),해외계열사계(매출액),국내매출액,해외매출액,매출액총계,year2,year,grpname2,매출사상/비,매출사금융/비금융,매출액Val,매입사상/비,매입사금융/비금융,grpname,grpcode,매출회사id,매입회사id
1457,에스케이,"SKChinaCompany,Ltd",99.0,1403811.0,77406.0,2829872.0,163941.0,2993813.0,2019,2018,에스케이,상장,비금융,2672412.0,NaN,NaN,sk,535,814261,-2147483648
1458,에스케이,"SKC&CBeijingCo.,Ltd",5335.0,1403811.0,77406.0,2829872.0,163941.0,2993813.0,2019,2018,에스케이,상장,비금융,2672412.0,NaN,NaN,sk,535,814261,-2147483648
1459,에스케이,ESSENCORELimited,4745.0,1403811.0,77406.0,2829872.0,163941.0,2993813.0,2019,2018,에스케이,상장,비금융,2672412.0,NaN,NaN,sk,535,814261,-2147483648
1460,에스케이,SKcomputerandcommunicationL.L.C,99.0,1403811.0,77406.0,2829872.0,163941.0,2993813.0,2019,2018,에스케이,상장,비금융,2672412.0,NaN,NaN,sk,535,814261,-2147483648
1461,에스케이,"SKMaterials(Jiangsu)Co.,Ltd",657.0,1403811.0,77406.0,2829872.0,163941.0,2993813.0,2019,2018,에스케이,상장,비금융,2672412.0,NaN,NaN,sk,535,814261,-2147483648
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12645,포스코우리이아이지글로벌사모투자,PWEFCooperatiefU.A,6032.0,NaN,6032.0,8.0,6032.0,6040.0,2019,2018,다우키움,비상장,금융,6040.0,NaN,NaN,daou,291,21047972,-2147483648
12648,한국전력우리스프랏글로벌해외자원개발제3차,COPAEurekaInc,415.0,NaN,415.0,1.0,415.0,416.0,2019,2018,다우키움,비상장,금융,416.0,NaN,NaN,daou,291,1903302530,-2147483648
12680,다우기술,다우재팬,339.0,58222.0,339.0,191862.0,407.0,192269.0,2019,2018,다우키움,상장,비금융,192269.0,NaN,NaN,daou,291,980650,-2147483648
12707,미래테크놀로지,다우재팬,87.0,76.0,87.0,22864.0,392.0,23256.0,2019,2018,다우키움,상장,비금융,23256.0,NaN,NaN,daou,291,180684,-2147483648


In [33]:
dfTrans20[dfTrans20['매입회사id'] == 5350000000]

,매출회사,매입회사,매출액,국내계열사계(매출액),해외계열사계(매출액),국내매출액,해외매출액,매출액총계,year2,year,grpname2,매출사상/비,매출사금융/비금융,매출액Val,매입사상/비,매입사금융/비금융,grpname,grpcode,매출회사id,매입회사id
1418,에스케이,"SKChinaCompany,Ltd",138.0,1401318.0,85604.0,3023603.0,221913.0,3245516.0,2020,2019,에스케이,상장,비금융,3245516.0,NaN,NaN,sk,535,814261,5350000000
1419,에스케이,"SKC&CBeijingCo.,Ltd",9726.0,1401318.0,85604.0,3023603.0,221913.0,3245516.0,2020,2019,에스케이,상장,비금융,3245516.0,NaN,NaN,sk,535,814261,5350000000
1420,에스케이,ESSENCORELimited,3713.0,1401318.0,85604.0,3023603.0,221913.0,3245516.0,2020,2019,에스케이,상장,비금융,3245516.0,NaN,NaN,sk,535,814261,5350000000
1421,에스케이,SKPharmtecoInc,251.0,1401318.0,85604.0,3023603.0,221913.0,3245516.0,2020,2019,에스케이,상장,비금융,3245516.0,NaN,NaN,sk,535,814261,5350000000
1422,에스케이,"SKSiltronAmerica,Inc",1204.0,1401318.0,85604.0,3023603.0,221913.0,3245516.0,2020,2019,에스케이,상장,비금융,3245516.0,NaN,NaN,sk,535,814261,5350000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13122,삼양사,"SamyangEngineeringPlastics(Shanghai)Co.,Ltd",20972.0,25247.0,35571.0,1137163.0,413638.0,1550801.0,2020,2019,삼양,상장,비금융,1551876.0,NaN,NaN,samyang,512,18078753,5350000000
13123,삼양사,"SamyangEPHungaryCo.,Ltd",14599.0,25247.0,35571.0,1137163.0,413638.0,1550801.0,2020,2019,삼양,상장,비금융,1551876.0,NaN,NaN,samyang,512,18078753,5350000000
13139,삼양데이타시스템,"SamyangEngineeringPlastics(Shanghai)Co.,Ltd",7.0,16509.0,147.0,60547.0,64.0,60611.0,2020,2019,삼양,비상장,비금융,60611.0,NaN,NaN,samyang,512,58010,5350000000
13140,삼양데이타시스템,"SamyangEPHungaryCo.,Ltd",2.0,16509.0,147.0,60547.0,64.0,60611.0,2020,2019,삼양,비상장,비금융,60611.0,NaN,NaN,samyang,512,58010,5350000000


In [34]:
dfTrans21[dfTrans21['매입회사id'] == 5350000000]

,매출회사,매입회사,매출액,국내계열사계(매출액),해외계열사계(매출액),국내매출액,해외매출액,매출액총계,year2,year,...,매출사금융/비금융,매출액Val,매입사상/비,매입사금융/비금융,grpname,grpcode,매출회사id,매입회사id,매출회사 실질지분율,매입회사 실질지분율
1428,에스케이,ESSENCORELimited,2369.0,1451273.0,101150.0,3365216.0,108768.0,3473984.0,2021,2020,...,비금융,3.470000e+12,NaN,NaN,sk,535,814261,5350000000,0.275832,NaN
1429,에스케이,"SKC&CBeijingCo.,Ltd",5599.0,1451273.0,101150.0,3365216.0,108768.0,3473984.0,2021,2020,...,비금융,3.470000e+12,NaN,NaN,sk,535,814261,5350000000,0.275832,NaN
1430,에스케이,SKBiotekIrelandLimited,3177.0,1451273.0,101150.0,3365216.0,108768.0,3473984.0,2021,2020,...,비금융,3.470000e+12,NaN,NaN,sk,535,814261,5350000000,0.275832,NaN
1431,에스케이,"SKBiotekUSA,Inc",82.0,1451273.0,101150.0,3365216.0,108768.0,3473984.0,2021,2020,...,비금융,3.470000e+12,NaN,NaN,sk,535,814261,5350000000,0.275832,NaN
1432,에스케이,SKPharmtecoInc.AlchemyAcquisitionCorp.,832.0,1451273.0,101150.0,3365216.0,108768.0,3473984.0,2021,2020,...,비금융,3.470000e+12,NaN,NaN,sk,535,814261,5350000000,0.275832,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14011,티씨이,TCEVINADENIMJOINTSTOCKCOMPANY,533.0,7.0,1323.0,4997.0,91944.0,96941.0,2021,2020,...,비금융,9.694100e+10,NaN,NaN,IS,3501209,13035598,5350000000,0.500000,NaN
14012,티씨이,TCEJEANSCO.LTD,790.0,7.0,1323.0,4997.0,91944.0,96941.0,2021,2020,...,비금융,9.694100e+10,NaN,NaN,IS,3501209,13035598,5350000000,0.500000,NaN
14172,조인스중앙,"JoongangilboUSA,Inc",38.0,18389.0,38.0,19069.0,38.0,19107.0,2021,2020,...,비금융,1.910700e+10,NaN,NaN,joongang,36006,459541,5350000000,0.406774,NaN
14202,중앙일보,"JoongangilboUSA,Inc",109.0,59992.0,109.0,272344.0,1844.0,274188.0,2021,2020,...,비금융,2.740000e+11,NaN,NaN,joongang,36006,340219,5350000000,0.756918,NaN


In [35]:
dfTrans19Dmstc = dfTrans19[(dfTrans19['매입회사id'] != -2147483648) & (dfTrans19['매출액'] > 0)]
len(dfTrans19Dmstc)

11196

In [36]:
dfTrans20Dmstc = dfTrans20[(dfTrans20['매입회사id'] != 5350000000) & (dfTrans20['매출액'] > 0)]
len(dfTrans20Dmstc)

11180

In [37]:
dfTrans21Dmstc = dfTrans21[(dfTrans21['매입회사id'] != 5350000000) & (dfTrans21['매출액'] > 0)]
len(dfTrans21Dmstc)

11843

In [74]:
dfAffilNum19 = pd.read_excel('../data/grpSumry/affilNumTtlAsset/affilNumTtlAsset2019.xlsx')
dfAffilNum19.head()

,chaebol,the_same_person_2019,n_affil_2019,ttl_asset_2019,rank_2019
0,삼성,이재용,62,414547,1
1,현대자동차,정몽구,53,223493,2
2,에스케이,최태원,111,218013,3
3,엘지,구광모,75,129616,4
4,롯데,신동빈,95,115339,5


In [75]:
dfAffilNum20 = pd.read_excel('../data/grpSumry/affilNumTtlAsset/affilNumTtlAsset2020.xlsx')
dfAffilNum20.head()

,chaebol,the_same_person_2020,n_affil_2020,ttl_asset_2020,rank_2020
0,삼성,이재용,59,424848,1
1,현대자동차,정몽구,54,234706,2
2,에스케이,최태원,125,225526,3
3,엘지,구광모,70,136967,4
4,롯데,신동빈,86,121524,5


In [40]:
dfAffilNum21 = pd.read_excel('../data/grpSumry/affilNumTtlAsset/affilNumTtlAsset2021.xlsx')
dfAffilNum21.head()

,chaebol,the_same_person_2021,n_affil_2021,ttl_asset_2021,rank_2021
0,삼성,이재용,59,457305,1
1,현대자동차,정의선,53,246084,2
2,에스케이,최태원,148,239530,3
3,엘지,구광모,70,151322,4
4,롯데,신동빈,86,117781,5


In [18]:
dfTrans18Dmstc = pd.read_excel("../data/transactions-report/2018 재벌 상품용역거래.xlsx")
dfTrans18Dmstc.head()

,year,year2,grpname,grpname2,grpcode,매출회사,매출회사id,매출사\n상/비,매입회사,매입회사id,매입사\n상/비,매출액,매출회사\n국내계열사계,매출회사\n해외계열사계,매출회사\n매출액총계,매출회사\n국내매출액,매출회사\n해외매출액,재벌순위,매출사 CFR\n(우선주포함),매출사 FAM\n(우선주포함)
0,2017,2018,DB,DB,365,DBInc.,352098,상장,DBInc.,352098,상장,0.0,43037.0,0,199530.0,123317.0,76213.0,36,0.401239,0.401239
1,2017,2018,DB,DB,365,DBInc.,352098,상장,DB금융서비스,23044292,비상장,0.0,43037.0,0,199530.0,123317.0,76213.0,36,0.401239,0.401239
2,2017,2018,DB,DB,365,DBInc.,352098,상장,DB금융투자,821640,상장,1424.0,43037.0,0,199530.0,123317.0,76213.0,36,0.401239,0.401239
3,2017,2018,DB,DB,365,DBInc.,352098,상장,DB라이텍,137656,상장,307.0,43037.0,0,199530.0,123317.0,76213.0,36,0.401239,0.401239
4,2017,2018,DB,DB,365,DBInc.,352098,상장,DB메탈,12097776,비상장,1566.0,43037.0,0,199530.0,123317.0,76213.0,36,0.401239,0.401239


In [21]:
dfTrans18Dmstc = dfTrans18Dmstc[dfTrans18Dmstc['매출액'] > 0]

In [57]:
dfAffilNum17 = pd.read_excel('../data/grpSumry/affilNumTtlAsset/affilNumTtlAsset2017.xlsx')
dfAffilNum17.head()

,chaebol,the_same_person_2017,n_affil_2017,ttl_asset_2017,rank_2017
0,삼성,이건희,62,363218,1
1,현대자동차,정몽구,53,218625,2
2,에스케이(주),최태원,96,170697,3
3,엘지,구본무,68,112326,4
4,롯데,신격호,90,110820,5


In [23]:
dfAffilNum18 = pd.read_excel('../data/grpSumry/affilNumTtlAsset/affilNumTtlAsset2018.xlsx')
dfAffilNum18.head()

,chaebol,the_same_person_2018,n_affil_2018,ttl_asset_2018,rank_2018
0,삼성,이재용,62,399479,1
1,현대자동차,정몽구,56,222654,2
2,에스케이,최태원,101,189531,3
3,엘지,구본무,70,123135,4
4,롯데,신동빈,107,116239,5


In [93]:
dfCmpnySumry18 = pd.read_excel('../data/cmpnySumry/cmpnySumry2018Kor.xlsx')
dfCmpnySumry18.head()

,공개년월,회계년,기업집단명,소속회사명,대표자,설립일,계열편입일,업종코드,영위업종,종업원수,...,기업공개일,자산총액,부채총액,자본총액,자본금,매출액,당기순이익,구분,법인등록번호,year2
0,201805,2017,DB,(주)동부월드,최경진,1989/05/15,1997/06/02,R91,스포츠 및 오락관련 서비스업,76,...,NaN,"188,988","57,318","131,670","161,454","10,894","-3,672",회생절차,1101110628086,2018
1,201805,2017,DB,디비씨앤에스자동차손해사정(주),문수원,2011/03/17,2011/04/05,K66,금융 및 보험 관련 서비스업,742,...,NaN,"10,602","6,887","3,715",200,"45,777","1,388",금융보험업,1101114558669,2018
2,201805,2017,DB,디비캐피탈(주),윤재인,1995/12/08,1995/12/08,K64,금융업,20,...,NaN,"186,265","103,745","82,520","62,014","11,477","2,344",금융보험업,1101111220641,2018
3,201805,2017,DB,디비금융투자(주),고원종,1982/12/20,1982/12/20,K66,금융 및 보험 관련 서비스업,968,...,1988/06/27,"5,401,797","4,808,879","592,918","212,232","806,256","2,591",금융보험업,1101110339956,2018
4,201805,2017,DB,디비생명보험(주),이태운,1989/04/14,1997/01/03,K65,보험 및 연금업,569,...,NaN,"10,663,512","10,024,347","639,165","193,331","2,378,149","29,215",금융보험업,1101110623606,2018


In [94]:
dfCmpnySumry19 = pd.read_excel('../data/cmpnySumry/cmpnySumry2019Kor.xlsx')
dfCmpnySumry19.head()

,공개년월,회계년,기업집단명,소속회사명,대표자,설립일,계열편입일,업종코드,영위업종,종업원수,...,자산총액,부채총액,자본총액,자본금,매출액,당기순이익,구분,법인등록번호,year2,rcmgCode
0,201905,2018,DB,디비생명보험(주),이태운,1989/04/14,1997/01/03,K65,보험 및 연금업,563,...,"10,887,144","10,296,289","590,855","193,331","2,180,825","24,637",금융보험업,1101110623606,2019,821420.0
1,201905,2018,DB,디비금융서비스(주),황원기,2014/01/13,2014/03/01,K66,금융 및 보험관련 서비스업,30,...,"6,057","2,460","3,597","14,001","13,311","-2,499",금융보험업,1101115314474,2019,23044292.0
2,201905,2018,DB,디비에프아이에스(주),강운식,2014/12/01,2018/06/01,J62,"컴퓨터 프로그래밍, 시스템 통합 및 관리업",217,...,"47,579","5,854","41,725","1,000","47,269","12,063",일반회사,1101115576727,2019,24038019.0
3,201905,2018,DB,디비금융투자(주),고원종,1982/12/20,1982/12/20,K66,금융 및 보험관련 서비스업,892,...,"5,428,689","4,794,353","634,336","212,232","725,156","46,854",금융보험업,1101110339956,2019,821640.0
4,201905,2018,DB,디비인베스트(주),최경진,2009/11/16,2010/01/04,M71,전문 서비스업,1,...,"14,912","17,943","-3,031","306,500",933,-267,일반회사,1101114220359,2019,16003715.0


In [95]:
dfCmpnySumry20 = pd.read_excel('../data/cmpnySumry/cmpnySumry2020Kor.xlsx')
dfCmpnySumry20.head()

,공개년월,회계년,기업집단명,소속회사명,대표자,설립일,계열편입일,업종코드,영위업종,종업원수,...,자산총액,부채총액,자본총액,자본금,매출액,당기순이익,구분,법인등록번호,year2,rcmgCode
0,202005,2019,넥슨,(유)가승개발,서춘식,2016/01/21,2016/01/21,R91,스포츠 및 오락관련 서비스업,3,...,"15,208","6,064","9,144",100,0,-465,일반회사,1345140003989,2020,1.802408e+09
1,202005,2019,지에스,(유)가승개발,서춘식,2016/01/21,2016/04/01,R91,스포츠 및 오락관련 서비스업,3,...,"15,208","6,064","9,144",100,NaN,-465,일반회사,1345140003989,2020,1.802408e+09
2,202005,2019,카카오,(유)매디슨카운티의다리,이명한,2016/11/16,2020/02/03,R90,"창작, 예술 및 여가관련 서비스업",0,...,11,28,-17,10,3,2,일반회사,1101140192605,2020,1.803409e+09
3,202005,2019,엘에스,(유)빌드윈문경태양광발전,김선철,2018/12/11,2019/02/01,D35,"전기, 가스, 증기 및 공기 조절 공급업",1,...,331,0,331,331,0,0,일반회사,1754140000582,2020,2.102105e+09
4,202005,2019,엘에스,(유)빌드윈인동태양광발전,김선철,2018/12/11,2019/02/01,D35,"전기, 가스, 증기 및 공기 조절 공급업",1,...,"4,883","4,140",743,756,0,-13,일반회사,1760140002014,2020,2.102105e+09


In [96]:
dfCmpnySumry21 = pd.read_excel('../data/cmpnySumry/cmpnySumry2021Kor.xlsx')
dfCmpnySumry21.head()

,공개년월,회계년,기업집단명,소속회사명,대표자,설립일,계열편입일,업종코드,영위업종,종업원수,...,자산총액,부채총액,자본총액,자본금,매출액,당기순이익,구분,법인등록번호,year2,rcmgCode
0,202105,2020,롯데,롯데쇼핑,강희태,19700702,19870401.0,G47,소매업; 자동차 제외,24963,...,2.490000e+13,1.530000e+13,9.600000e+12,1.410000e+11,8.710000e+12,-1.030000e+12,일반회사,1100000000000,2021,620068
1,202105,2020,지에스,지에스건설,임병용,19691219,20050404.0,F41,종합 건설업,6928,...,1.190000e+13,7.450000e+12,4.420000e+12,4.040000e+11,8.890000e+12,3.270000e+11,일반회사,1100000000000,2021,500143
2,202105,2020,SM,삼라,조유선,19690826,20040501.0,F41,종합 건설업,12,...,6.240000e+11,2.720000e+11,3.520000e+11,1.685000e+10,1.490000e+11,7.785100e+10,일반회사,1100000000000,2021,500429
3,202105,2020,한화,한화,옥경석 외,19521028,19870401.0,C20,화학 물질 및 화학제품 제조업; 의약품 제외,5026,...,7.360000e+12,3.870000e+12,3.490000e+12,4.900000e+11,4.000000e+12,1.290000e+11,일반회사,1100000000000,2021,350893
4,202105,2020,동원,동원산업,이명우,19690416,20030128.0,A03,어업,761,...,1.630000e+12,8.800000e+11,7.540000e+11,1.838800e+10,7.020000e+11,7.895000e+10,일반회사,1100000000000,2021,100030


In [106]:
dfCmpnySumry18 = dfCmpnySumry18[dfCmpnySumry18['기업집단명'].isin(dfAffilNum18['chaebol'])]
dfCmpnySumry18['기업집단명'].nunique()

52

In [109]:
dfCmpnySumry19 = dfCmpnySumry19[dfCmpnySumry19['기업집단명'].isin(dfAffilNum19['chaebol'])]
dfCmpnySumry19['기업집단명'].nunique()

47

In [110]:
dfCmpnySumry20 = dfCmpnySumry20[dfCmpnySumry20['기업집단명'].isin(dfAffilNum20['chaebol'])]
dfCmpnySumry20['기업집단명'].nunique()

55

In [107]:
dfCmpnySumry21 = dfCmpnySumry21[dfCmpnySumry21['기업집단명'].isin(dfAffilNum21['chaebol'])]
dfCmpnySumry21['기업집단명'].nunique()

60

In [112]:
dfCmpnySumry18['소속회사명'].nunique()

1914

In [113]:
dfCmpnySumry19['소속회사명'].nunique()

1729

In [114]:
dfCmpnySumry20['소속회사명'].nunique()

2096

In [115]:
dfCmpnySumry21['소속회사명'].nunique()

2410

In [125]:
dfTrans19Dmstc[dfTrans19Dmstc['매출회사상/비'] == '상장']

,매출회사,매입회사,매출액,국내계열사계(매출액),해외계열사계(매출액),국내매출액,해외매출액,매출액총계,year2,year,grpname2,매출사상/비,매출사금융/비금융,매출액Val,매입사상/비,매입사금융/비금융,grpname,grpcode,매출회사id,매입회사id
0,삼성전자,삼성물산,78676.0,2269950.0,149436938.0,16821275.0,153560595.0,170381870.0,2019,2018,삼성,상장,비금융,170381870.0,상장,비금융,samsung,511,380725,810053
1,삼성전자,삼성바이오에피스,1.0,2269950.0,149436938.0,16821275.0,153560595.0,170381870.0,2019,2018,삼성,상장,비금융,170381870.0,비상장,비금융,samsung,511,380725,18082125
2,삼성전자,삼성디스플레이,164404.0,2269950.0,149436938.0,16821275.0,153560595.0,170381870.0,2019,2018,삼성,상장,비금융,170381870.0,비상장,비금융,samsung,511,380725,18082126
3,삼성전자,세메스,5.0,2269950.0,149436938.0,16821275.0,153560595.0,170381870.0,2019,2018,삼성,상장,비금융,170381870.0,비상장,비금융,samsung,511,380725,314528
4,삼성전자,삼성전자서비스,456109.0,2269950.0,149436938.0,16821275.0,153560595.0,170381870.0,2019,2018,삼성,상장,비금융,170381870.0,비상장,비금융,samsung,511,380725,952133


## 기업수

In [124]:
dfCmpnySumry18[~dfCmpnySumry18['기업공개일'].isna()]['소속회사명'].nunique()

241

In [205]:
dfCmpnySumry18[dfCmpnySumry18['기업공개일'].isna()]['소속회사명'].nunique()

1673

In [130]:
dfCmpnySumry19[~dfCmpnySumry19['기업공개일'].isna()]['소속회사명'].nunique()

210

In [206]:
dfCmpnySumry19[dfCmpnySumry19['기업공개일'].isna()]['소속회사명'].nunique()

1519

In [131]:
dfCmpnySumry20[~dfCmpnySumry20['기업공개일'].isna()]['소속회사명'].nunique()

256

In [207]:
dfCmpnySumry20[dfCmpnySumry20['기업공개일'].isna()]['소속회사명'].nunique()

1840

In [132]:
dfCmpnySumry21[~dfCmpnySumry21['기업공개일'].isna()]['소속회사명'].nunique()

258

In [208]:
dfCmpnySumry21[dfCmpnySumry21['기업공개일'].isna()]['소속회사명'].nunique()

2152

## 상장비율

In [139]:
dfCmpnySumry18[~dfCmpnySumry18['기업공개일'].isna()]['소속회사명'].nunique() / dfCmpnySumry18['소속회사명'].nunique() 

0.12591431556948798

In [140]:
dfCmpnySumry19[~dfCmpnySumry19['기업공개일'].isna()]['소속회사명'].nunique() / dfCmpnySumry19['소속회사명'].nunique()

0.1214574898785425

In [141]:
dfCmpnySumry20[~dfCmpnySumry20['기업공개일'].isna()]['소속회사명'].nunique() / dfCmpnySumry20['소속회사명'].nunique()

0.12213740458015267

In [142]:
dfCmpnySumry21[~dfCmpnySumry21['기업공개일'].isna()]['소속회사명'].nunique() / dfCmpnySumry21['소속회사명'].nunique()

0.10705394190871369

## 평균거래계열사수

In [145]:
dfTrans18Dmstc.head()

,year,year2,grpname,grpname2,grpcode,매출회사,매출회사id,매출사\n상/비,매입회사,매입회사id,매입사\n상/비,매출액,매출회사\n국내계열사계,매출회사\n해외계열사계,매출회사\n매출액총계,매출회사\n국내매출액,매출회사\n해외매출액,재벌순위,매출사 CFR\n(우선주포함),매출사 FAM\n(우선주포함)
2,2017,2018,DB,DB,365,DBInc.,352098,상장,DB금융투자,821640,상장,1424.0,43037.0,0,199530.0,123317.0,76213.0,36,0.401239,0.401239
3,2017,2018,DB,DB,365,DBInc.,352098,상장,DB라이텍,137656,상장,307.0,43037.0,0,199530.0,123317.0,76213.0,36,0.401239,0.401239
4,2017,2018,DB,DB,365,DBInc.,352098,상장,DB메탈,12097776,비상장,1566.0,43037.0,0,199530.0,123317.0,76213.0,36,0.401239,0.401239
5,2017,2018,DB,DB,365,DBInc.,352098,상장,DB생명보험,821420,비상장,5291.0,43037.0,0,199530.0,123317.0,76213.0,36,0.401239,0.401239
6,2017,2018,DB,DB,365,DBInc.,352098,상장,DB손해보험,870099,상장,13298.0,43037.0,0,199530.0,123317.0,76213.0,36,0.401239,0.401239


In [197]:
len(dfTrans18Dmstc) /\
dfCmpnySumry18['소속회사명'].nunique()

5.377742946708464

In [146]:
len(dfTrans18Dmstc[dfTrans18Dmstc['매출사\n상/비'] == '상장']) /\
dfCmpnySumry18[~dfCmpnySumry18['기업공개일'].isna()]['소속회사명'].nunique()

14.867219917012449

In [196]:
len(dfTrans18Dmstc[dfTrans18Dmstc['매출사\n상/비'] == '비상장']) /\
dfCmpnySumry18[dfCmpnySumry18['기업공개일'].isna()]['소속회사명'].nunique()

4.010759115361626

In [198]:
len(dfTrans19Dmstc) /\
dfCmpnySumry19['소속회사명'].nunique()

6.47541931752458

In [148]:
len(dfTrans19Dmstc[dfTrans19Dmstc['매출사상/비'] == '상장']) /\
dfCmpnySumry19[~dfCmpnySumry19['기업공개일'].isna()]['소속회사명'].nunique()

18.97142857142857

In [199]:
len(dfTrans19Dmstc[dfTrans19Dmstc['매출사상/비'] == '비상장']) /\
dfCmpnySumry19[dfCmpnySumry19['기업공개일'].isna()]['소속회사명'].nunique()

4.720210664911126

In [200]:
len(dfTrans20Dmstc) /\
dfCmpnySumry20['소속회사명'].nunique()

5.333969465648855

In [150]:
len(dfTrans20Dmstc[dfTrans20Dmstc['매출사상/비'] == '상장']) /\
dfCmpnySumry20[~dfCmpnySumry20['기업공개일'].isna()]['소속회사명'].nunique()

15.39453125

In [201]:
len(dfTrans20Dmstc[dfTrans20Dmstc['매출사상/비'] == '비상장']) /\
dfCmpnySumry20[dfCmpnySumry20['기업공개일'].isna()]['소속회사명'].nunique()

3.9326086956521737

In [203]:
len(dfTrans21Dmstc) /\
dfCmpnySumry21['소속회사명'].nunique()

4.914107883817428

In [151]:
len(dfTrans21Dmstc[dfTrans21Dmstc['매출사상/비'] == '상장']) /\
dfCmpnySumry21[~dfCmpnySumry21['기업공개일'].isna()]['소속회사명'].nunique()

15.717054263565892

In [204]:
len(dfTrans21Dmstc[dfTrans21Dmstc['매출사상/비'] == '비상장']) /\
dfCmpnySumry21[dfCmpnySumry21['기업공개일'].isna()]['소속회사명'].nunique()

3.579925650557621

## 평균내부거래금액

In [192]:
dfTrans18Dmstc['매출액'].sum() /\
dfCmpnySumry18['소속회사명'].nunique()

89177.05471786835

In [153]:
dfTrans18Dmstc[dfTrans18Dmstc['매출사\n상/비'] == '상장']['매출액'].sum() /\
dfCmpnySumry18[~dfCmpnySumry18['기업공개일'].isna()]['소속회사명'].nunique()

309381.1704979253

In [191]:
dfTrans18Dmstc[dfTrans18Dmstc['매출사\n상/비'] == '비상장']['매출액'].sum() /\
dfCmpnySumry18[dfCmpnySumry18['기업공개일'].isna()]['소속회사명'].nunique()

57456.079282725645

In [193]:
dfTrans19Dmstc['매출액'].sum() /\
dfCmpnySumry19['소속회사명'].nunique()

104231.45515905149

In [154]:
dfTrans19Dmstc[dfTrans19Dmstc['매출사상/비'] == '상장']['매출액'].sum() /\
dfCmpnySumry19[~dfCmpnySumry19['기업공개일'].isna()]['소속회사명'].nunique()

377488.03652380954

In [189]:
dfTrans19Dmstc[dfTrans19Dmstc['매출사상/비'] == '비상장']['매출액'].sum() /\
dfCmpnySumry19[dfCmpnySumry19['기업공개일'].isna()]['소속회사명'].nunique()

66291.31816984859

In [194]:
dfTrans20Dmstc['매출액'].sum() /\
dfCmpnySumry20['소속회사명'].nunique()

83745.20840648856

In [155]:
dfTrans20Dmstc[dfTrans20Dmstc['매출사상/비'] == '상장']['매출액'].sum() /\
dfCmpnySumry20[~dfCmpnySumry20['기업공개일'].isna()]['소속회사명'].nunique()

313778.3273828125

In [188]:
dfTrans20Dmstc[dfTrans20Dmstc['매출사상/비'] == '비상장']['매출액'].sum() /\
dfCmpnySumry20[dfCmpnySumry20['기업공개일'].isna()]['소속회사명'].nunique()

51738.08207065218

In [195]:
dfTrans21Dmstc['매출액'].sum() /\
dfCmpnySumry21['소속회사명'].nunique()

68118.61605850623

In [156]:
dfTrans21Dmstc[dfTrans21Dmstc['매출사상/비'] == '상장']['매출액'].sum() /\
dfCmpnySumry21[~dfCmpnySumry21['기업공개일'].isna()]['소속회사명'].nunique()

300942.37004263565

In [187]:
dfTrans21Dmstc[dfTrans21Dmstc['매출사상/비'] == '비상장']['매출액'].sum() /\
dfCmpnySumry21[dfCmpnySumry21['기업공개일'].isna()]['소속회사명'].nunique()

38886.16274628253

## 총매출액 대비 내부 거래 비중

In [158]:
dfTrans18Dmstc[dfTrans18Dmstc['매출사\n상/비'] == '상장']['매출액'].sum()

74560862.09

In [164]:
dfCmpnySumry18['매출액'] = dfCmpnySumry18['매출액'].replace(',', '', regex=True).astype(float)
dfCmpnySumry19['매출액'] = dfCmpnySumry19['매출액'].replace(',', '', regex=True).astype(float)
dfCmpnySumry20['매출액'] = dfCmpnySumry20['매출액'].replace(',', '', regex=True).astype(float)
dfCmpnySumry21['매출액'] = dfCmpnySumry21['매출액'].replace(',', '', regex=True).astype(float)

In [176]:
dfTrans18Dmstc['매출액'].sum() /\
dfCmpnySumry18['매출액'].sum()

0.12403513724952457

In [165]:
dfTrans18Dmstc[dfTrans18Dmstc['매출사\n상/비'] == '상장']['매출액'].sum() /\
dfCmpnySumry18[~dfCmpnySumry18['기업공개일'].isna()]['매출액'].sum()

0.0782614507175856

In [186]:
dfTrans18Dmstc[dfTrans18Dmstc['매출사\n상/비'] == '비상장']['매출액'].sum() /\
dfCmpnySumry18[dfCmpnySumry18['기업공개일'].isna()]['매출액'].sum()

0.22703637075159872

In [177]:
dfTrans19Dmstc['매출액'].sum() /\
dfCmpnySumry19['매출액'].sum()

0.13610687872554067

In [168]:
dfTrans19Dmstc[dfTrans19Dmstc['매출사상/비'] == '상장']['매출액'].sum() /\
dfCmpnySumry19[~dfCmpnySumry19['기업공개일'].isna()]['매출액'].sum()

0.08722634434487488

In [184]:
dfTrans19Dmstc[dfTrans19Dmstc['매출사상/비'] == '비상장']['매출액'].sum() /\
dfCmpnySumry19[dfCmpnySumry19['기업공개일'].isna()]['매출액'].sum()

0.24248743815576534

In [178]:
dfTrans20Dmstc['매출액'].sum() /\
dfCmpnySumry20['매출액'].sum()

0.12519751803095414

In [169]:
dfTrans20Dmstc[dfTrans20Dmstc['매출사상/비'] == '상장']['매출액'].sum() /\
dfCmpnySumry20[~dfCmpnySumry20['기업공개일'].isna()]['매출액'].sum()

0.08403641724358478

In [183]:
dfTrans20Dmstc[dfTrans20Dmstc['매출사상/비'] == '비상장']['매출액'].sum() /\
dfCmpnySumry20[dfCmpnySumry20['기업공개일'].isna()]['매출액'].sum()

0.21337115894153658

In [180]:
dfTrans21Dmstc['매출액'].sum() /\
dfCmpnySumry21['매출액'].sum() * 1000000

0.11843155744759018

In [175]:
dfTrans21Dmstc[dfTrans21Dmstc['매출사상/비'] == '상장']['매출액'].sum() /\
dfCmpnySumry21[~dfCmpnySumry21['기업공개일'].isna()]['매출액'].sum() * 1000000

0.07972019573023983

In [182]:
dfTrans21Dmstc[dfTrans21Dmstc['매출사상/비'] == '비상장']['매출액'].sum() /\
dfCmpnySumry21[dfCmpnySumry21['기업공개일'].isna()]['매출액'].sum() * 1000000

0.20300524247199242

In [97]:
dfTrans18Dmstc.head()

,year,year2,grpname,grpname2,grpcode,매출회사,매출회사id,매출사\n상/비,매입회사,매입회사id,매입사\n상/비,매출액,매출회사\n국내계열사계,매출회사\n해외계열사계,매출회사\n매출액총계,매출회사\n국내매출액,매출회사\n해외매출액,재벌순위,매출사 CFR\n(우선주포함),매출사 FAM\n(우선주포함)
2,2017,2018,DB,DB,365,DBInc.,352098,상장,DB금융투자,821640,상장,1424.0,43037.0,0,199530.0,123317.0,76213.0,36,0.401239,0.401239
3,2017,2018,DB,DB,365,DBInc.,352098,상장,DB라이텍,137656,상장,307.0,43037.0,0,199530.0,123317.0,76213.0,36,0.401239,0.401239
4,2017,2018,DB,DB,365,DBInc.,352098,상장,DB메탈,12097776,비상장,1566.0,43037.0,0,199530.0,123317.0,76213.0,36,0.401239,0.401239
5,2017,2018,DB,DB,365,DBInc.,352098,상장,DB생명보험,821420,비상장,5291.0,43037.0,0,199530.0,123317.0,76213.0,36,0.401239,0.401239
6,2017,2018,DB,DB,365,DBInc.,352098,상장,DB손해보험,870099,상장,13298.0,43037.0,0,199530.0,123317.0,76213.0,36,0.401239,0.401239


# 10대 재벌

In [210]:
dfAffilNum18['chaebol'][:10]

0       삼성
1    현대자동차
2     에스케이
3       엘지
4       롯데
5      지에스
6       한화
7    현대중공업
8      신세계
9       두산
Name: chaebol, dtype: object

In [212]:
dfTrans18Dmstc10Big = dfTrans18Dmstc[dfTrans18Dmstc['grpname2'].isin(dfAffilNum18['chaebol'][:10])]
dfTrans18Dmstc10Big['grpname2'].unique()

array(['두산', '롯데', '삼성', '신세계', '에스케이', '엘지', '지에스', '한화', '현대자동차',
       '현대중공업'], dtype=object)

In [213]:
dfCmpnySumry18Big10 = dfCmpnySumry18[dfCmpnySumry18['기업집단명'].isin(dfAffilNum18['chaebol'][:10])]
dfCmpnySumry18Big10['기업집단명'].unique()

array(['두산', '롯데', '삼성', '신세계', '에스케이', '엘지', '지에스', '한화', '현대자동차',
       '현대중공업'], dtype=object)

In [214]:
dfTrans18Dmstc10Big = dfTrans18Dmstc[dfTrans18Dmstc['grpname2'].isin(dfAffilNum18['chaebol'][:10])]
dfCmpnySumry18Big10 = dfCmpnySumry18[dfCmpnySumry18['기업집단명'].isin(dfAffilNum18['chaebol'][:10])]
dfTrans18Dmstc5Big = dfTrans18Dmstc[dfTrans18Dmstc['grpname2'].isin(dfAffilNum18['chaebol'][:5])]
dfCmpnySumry18Big5 = dfCmpnySumry18[dfCmpnySumry18['기업집단명'].isin(dfAffilNum18['chaebol'][:5])]

In [215]:
dfTrans19Dmstc10Big = dfTrans19Dmstc[dfTrans19Dmstc['grpname2'].isin(dfAffilNum19['chaebol'][:10])]
dfCmpnySumry19Big10 = dfCmpnySumry19[dfCmpnySumry19['기업집단명'].isin(dfAffilNum19['chaebol'][:10])]
dfTrans19Dmstc5Big = dfTrans19Dmstc[dfTrans19Dmstc['grpname2'].isin(dfAffilNum19['chaebol'][:5])]
dfCmpnySumry19Big5 = dfCmpnySumry19[dfCmpnySumry19['기업집단명'].isin(dfAffilNum19['chaebol'][:5])]

In [217]:
dfTrans20Dmstc10Big = dfTrans20Dmstc[dfTrans20Dmstc['grpname2'].isin(dfAffilNum20['chaebol'][:10])]
dfCmpnySumry20Big10 = dfCmpnySumry20[dfCmpnySumry20['기업집단명'].isin(dfAffilNum20['chaebol'][:10])]
dfTrans20Dmstc5Big = dfTrans20Dmstc[dfTrans20Dmstc['grpname2'].isin(dfAffilNum20['chaebol'][:5])]
dfCmpnySumry20Big5 = dfCmpnySumry20[dfCmpnySumry20['기업집단명'].isin(dfAffilNum20['chaebol'][:5])]

In [218]:
dfTrans21Dmstc10Big = dfTrans21Dmstc[dfTrans21Dmstc['grpname2'].isin(dfAffilNum21['chaebol'][:10])]
dfCmpnySumry21Big10 = dfCmpnySumry21[dfCmpnySumry21['기업집단명'].isin(dfAffilNum21['chaebol'][:10])]
dfTrans21Dmstc5Big = dfTrans21Dmstc[dfTrans21Dmstc['grpname2'].isin(dfAffilNum21['chaebol'][:5])]
dfCmpnySumry21Big5 = dfCmpnySumry21[dfCmpnySumry21['기업집단명'].isin(dfAffilNum21['chaebol'][:5])]

## 기업수

In [219]:
dfCmpnySumry18Big10[~dfCmpnySumry18Big10['기업공개일'].isna()]['소속회사명'].nunique()

98

In [221]:
dfCmpnySumry18Big10[dfCmpnySumry18Big10['기업공개일'].isna()]['소속회사명'].nunique()

535

In [222]:
dfCmpnySumry19Big10[~dfCmpnySumry19Big10['기업공개일'].isna()]['소속회사명'].nunique()

96

In [223]:
dfCmpnySumry19Big10[dfCmpnySumry19Big10['기업공개일'].isna()]['소속회사명'].nunique()

539

In [224]:
dfCmpnySumry20Big10[~dfCmpnySumry20Big10['기업공개일'].isna()]['소속회사명'].nunique()

105

In [225]:
dfCmpnySumry20Big10[dfCmpnySumry20Big10['기업공개일'].isna()]['소속회사명'].nunique()

589

In [227]:
dfCmpnySumry21Big10[~dfCmpnySumry21Big10['기업공개일'].isna()]['소속회사명'].nunique()

105

In [226]:
dfCmpnySumry21Big10[dfCmpnySumry21Big10['기업공개일'].isna()]['소속회사명'].nunique()

628

## 상장비율

In [228]:
dfCmpnySumry18Big10[~dfCmpnySumry18Big10['기업공개일'].isna()]['소속회사명'].nunique() /\
dfCmpnySumry18Big10['소속회사명'].nunique() 

0.15481832543443919

In [140]:
dfCmpnySumry19Big10[~dfCmpnySumry19Big10['기업공개일'].isna()]['소속회사명'].nunique() /\
dfCmpnySumry19Big10['소속회사명'].nunique()

0.1214574898785425

In [229]:
dfCmpnySumry20Big10[~dfCmpnySumry20Big10['기업공개일'].isna()]['소속회사명'].nunique() /\
dfCmpnySumry20Big10['소속회사명'].nunique()

0.15129682997118155

In [230]:
dfCmpnySumry21Big10[~dfCmpnySumry21Big10['기업공개일'].isna()]['소속회사명'].nunique() /\
dfCmpnySumry21Big10['소속회사명'].nunique()

0.1432469304229195

## 평균거래계열사수

In [231]:
len(dfTrans18Dmstc10Big) /\
dfCmpnySumry18Big10['소속회사명'].nunique()

9.437598736176936

In [232]:
len(dfTrans18Dmstc10Big[dfTrans18Dmstc10Big['매출사\n상/비'] == '상장']) /\
dfCmpnySumry18Big10[~dfCmpnySumry18Big10['기업공개일'].isna()]['소속회사명'].nunique()

22.93877551020408

In [233]:
len(dfTrans18Dmstc10Big[dfTrans18Dmstc10Big['매출사\n상/비'] == '비상장']) /\
dfCmpnySumry18Big10[dfCmpnySumry18Big10['기업공개일'].isna()]['소속회사명'].nunique()

6.964485981308411

In [234]:
len(dfTrans19Dmstc10Big) /\
dfCmpnySumry19Big10['소속회사명'].nunique()

10.171653543307087

In [235]:
len(dfTrans19Dmstc10Big[dfTrans19Dmstc10Big['매출사상/비'] == '상장']) /\
dfCmpnySumry19Big10[~dfCmpnySumry19Big10['기업공개일'].isna()]['소속회사명'].nunique()

26.572916666666668

In [236]:
len(dfTrans19Dmstc10Big[dfTrans19Dmstc10Big['매출사상/비'] == '비상장']) /\
dfCmpnySumry19Big10[dfCmpnySumry19Big10['기업공개일'].isna()]['소속회사명'].nunique()

7.244897959183674

In [237]:
len(dfTrans20Dmstc10Big) /\
dfCmpnySumry20Big10['소속회사명'].nunique()

9.32420749279539

In [238]:
len(dfTrans20Dmstc10Big[dfTrans20Dmstc10Big['매출사상/비'] == '상장']) /\
dfCmpnySumry20Big10[~dfCmpnySumry20Big10['기업공개일'].isna()]['소속회사명'].nunique()

24.495238095238093

In [239]:
len(dfTrans20Dmstc10Big[dfTrans20Dmstc10Big['매출사상/비'] == '비상장']) /\
dfCmpnySumry20Big10[dfCmpnySumry20Big10['기업공개일'].isna()]['소속회사명'].nunique()

6.619694397283531

In [240]:
len(dfTrans21Dmstc10Big) /\
dfCmpnySumry21Big10['소속회사명'].nunique()

8.881309686221009

In [241]:
len(dfTrans21Dmstc10Big[dfTrans21Dmstc10Big['매출사상/비'] == '상장']) /\
dfCmpnySumry21Big10[~dfCmpnySumry21Big10['기업공개일'].isna()]['소속회사명'].nunique()

24.83809523809524

In [242]:
len(dfTrans21Dmstc10Big[dfTrans21Dmstc10Big['매출사상/비'] == '비상장']) /\
dfCmpnySumry21Big10[dfCmpnySumry21Big10['기업공개일'].isna()]['소속회사명'].nunique()

6.1050955414012735

## 평균내부거래금액

In [243]:
dfTrans18Dmstc10Big['매출액'].sum() /\
dfCmpnySumry18Big10['소속회사명'].nunique()

224344.33807266984

In [244]:
dfTrans18Dmstc10Big[dfTrans18Dmstc10Big['매출사\n상/비'] == '상장']['매출액'].sum() /\
dfCmpnySumry18Big10[~dfCmpnySumry18Big10['기업공개일'].isna()]['소속회사명'].nunique()

637309.9489795918

In [245]:
dfTrans18Dmstc10Big[dfTrans18Dmstc10Big['매출사\n상/비'] == '비상장']['매출액'].sum() /\
dfCmpnySumry18Big10[dfCmpnySumry18Big10['기업공개일'].isna()]['소속회사명'].nunique()

148698.30093457943

In [246]:
dfTrans19Dmstc10Big['매출액'].sum() /\
dfCmpnySumry19Big10['소속회사명'].nunique()

242071.67642519684

In [247]:
dfTrans19Dmstc10Big[dfTrans19Dmstc10Big['매출사상/비'] == '상장']['매출액'].sum() /\
dfCmpnySumry19Big10[~dfCmpnySumry19Big10['기업공개일'].isna()]['소속회사명'].nunique()

702813.8805208333

In [248]:
dfTrans19Dmstc10Big[dfTrans19Dmstc10Big['매출사상/비'] == '비상장']['매출액'].sum() /\
dfCmpnySumry19Big10[dfCmpnySumry19Big10['기업공개일'].isna()]['소속회사명'].nunique()

160005.33580705008

In [249]:
dfTrans20Dmstc10Big['매출액'].sum() /\
dfCmpnySumry20Big10['소속회사명'].nunique()

218405.34250720462

In [250]:
dfTrans20Dmstc10Big[dfTrans20Dmstc10Big['매출사상/비'] == '상장']['매출액'].sum() /\
dfCmpnySumry20Big10[~dfCmpnySumry20Big10['기업공개일'].isna()]['소속회사명'].nunique()

673349.3333333334

In [251]:
dfTrans20Dmstc10Big[dfTrans20Dmstc10Big['매출사상/비'] == '비상장']['매출액'].sum() /\
dfCmpnySumry20Big10[dfCmpnySumry20Big10['기업공개일'].isna()]['소속회사명'].nunique()

137303.2728353141

In [252]:
dfTrans21Dmstc10Big['매출액'].sum() /\
dfCmpnySumry21Big10['소속회사명'].nunique()

188750.2545702592

In [253]:
dfTrans21Dmstc10Big[dfTrans21Dmstc10Big['매출사상/비'] == '상장']['매출액'].sum() /\
dfCmpnySumry21Big10[~dfCmpnySumry21Big10['기업공개일'].isna()]['소속회사명'].nunique()

655406.2095238095

In [254]:
dfTrans21Dmstc10Big[dfTrans21Dmstc10Big['매출사상/비'] == '비상장']['매출액'].sum() /\
dfCmpnySumry21Big10[dfCmpnySumry21Big10['기업공개일'].isna()]['소속회사명'].nunique()

106319.9101910828

## 총매출액 대비 내부 거래 비중

In [255]:
dfTrans18Dmstc10Big[dfTrans18Dmstc10Big['매출사\n상/비'] == '상장']['매출액'].sum()

62456375.0

In [256]:
dfCmpnySumry18Big10['매출액'] = dfCmpnySumry18Big10['매출액'].replace(',', '', regex=True).astype(float)
dfCmpnySumry19Big10['매출액'] = dfCmpnySumry19Big10['매출액'].replace(',', '', regex=True).astype(float)
dfCmpnySumry20Big10['매출액'] = dfCmpnySumry20Big10['매출액'].replace(',', '', regex=True).astype(float)
dfCmpnySumry21Big10['매출액'] = dfCmpnySumry21Big10['매출액'].replace(',', '', regex=True).astype(float)

/var/folders/xw/1rpcyks10ps27v4w_vlbb6s00000gn/T/ipykernel_37869/3917260575.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfCmpnySumry18Big10['매출액'] = dfCmpnySumry18Big10['매출액'].replace(',', '', regex=True).astype(float)
/var/folders/xw/1rpcyks10ps27v4w_vlbb6s00000gn/T/ipykernel_37869/3917260575.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfCmpnySumry19Big10['매출액'] = dfCmpnySumry19Big10['매출액'].replace(',', '', regex=True).astype(float)
/var/folders/xw/1rpcyks10ps27v4w_vlbb6s00000gn/T/ipykernel

In [257]:
dfTrans18Dmstc10Big['매출액'].sum() /\
dfCmpnySumry18Big10['매출액'].sum()

0.13640852179098226

In [258]:
dfTrans18Dmstc10Big[dfTrans18Dmstc10Big['매출사\n상/비'] == '상장']['매출액'].sum() /\
dfCmpnySumry18Big10[~dfCmpnySumry18Big10['기업공개일'].isna()]['매출액'].sum()

0.08483720166654059

In [259]:
dfTrans18Dmstc10Big[dfTrans18Dmstc10Big['매출사\n상/비'] == '비상장']['매출액'].sum() /\
dfCmpnySumry18Big10[dfCmpnySumry18Big10['기업공개일'].isna()]['매출액'].sum()

0.2609402271768699

In [260]:
dfTrans19Dmstc10Big['매출액'].sum() /\
dfCmpnySumry19Big10['매출액'].sum()

0.14724977358513205

In [261]:
dfTrans19Dmstc10Big[dfTrans19Dmstc10Big['매출사상/비'] == '상장']['매출액'].sum() /\
dfCmpnySumry19Big10[~dfCmpnySumry19Big10['기업공개일'].isna()]['매출액'].sum()

0.09129619104595564

In [262]:
dfTrans19Dmstc10Big[dfTrans19Dmstc10Big['매출사상/비'] == '비상장']['매출액'].sum() /\
dfCmpnySumry19Big10[dfCmpnySumry19Big10['기업공개일'].isna()]['매출액'].sum()

0.28286980622854146

In [263]:
dfTrans20Dmstc10Big['매출액'].sum() /\
dfCmpnySumry20Big10['매출액'].sum()

0.14163446160198662

In [264]:
dfTrans20Dmstc10Big[dfTrans20Dmstc10Big['매출사상/비'] == '상장']['매출액'].sum() /\
dfCmpnySumry20Big10[~dfCmpnySumry20Big10['기업공개일'].isna()]['매출액'].sum()

0.09468821474050394

In [265]:
dfTrans20Dmstc10Big[dfTrans20Dmstc10Big['매출사상/비'] == '비상장']['매출액'].sum() /\
dfCmpnySumry20Big10[dfCmpnySumry20Big10['기업공개일'].isna()]['매출액'].sum()

0.24999441318400784

In [266]:
dfTrans21Dmstc10Big['매출액'].sum() /\
dfCmpnySumry21Big10['매출액'].sum() * 1000000

0.13400424107916992

In [267]:
dfTrans21Dmstc10Big[dfTrans21Dmstc10Big['매출사상/비'] == '상장']['매출액'].sum() /\
dfCmpnySumry21Big10[~dfCmpnySumry21Big10['기업공개일'].isna()]['매출액'].sum() * 1000000

0.09060785612925931

In [268]:
dfTrans21Dmstc10Big[dfTrans21Dmstc10Big['매출사상/비'] == '비상장']['매출액'].sum() /\
dfCmpnySumry21Big10[dfCmpnySumry21Big10['기업공개일'].isna()]['매출액'].sum() * 1000000

0.2446208732413941

# 5대 재벌

## 기업수

In [269]:
dfCmpnySumry18Big5[~dfCmpnySumry18Big5['기업공개일'].isna()]['소속회사명'].nunique()

66

In [270]:
dfCmpnySumry18Big5[dfCmpnySumry18Big5['기업공개일'].isna()]['소속회사명'].nunique()

329

In [271]:
dfCmpnySumry19Big5[~dfCmpnySumry19Big5['기업공개일'].isna()]['소속회사명'].nunique()

66

In [272]:
dfCmpnySumry19Big5[dfCmpnySumry19Big5['기업공개일'].isna()]['소속회사명'].nunique()

329

In [273]:
dfCmpnySumry20Big5[~dfCmpnySumry20Big5['기업공개일'].isna()]['소속회사명'].nunique()

70

In [274]:
dfCmpnySumry20Big5[dfCmpnySumry20Big5['기업공개일'].isna()]['소속회사명'].nunique()

323

In [275]:
dfCmpnySumry21Big5[~dfCmpnySumry21Big5['기업공개일'].isna()]['소속회사명'].nunique()

70

In [276]:
dfCmpnySumry21Big5[dfCmpnySumry21Big5['기업공개일'].isna()]['소속회사명'].nunique()

345

## 상장비율

In [277]:
dfCmpnySumry18Big5[~dfCmpnySumry18Big5['기업공개일'].isna()]['소속회사명'].nunique() /\
dfCmpnySumry18Big5['소속회사명'].nunique() 

0.1670886075949367

In [278]:
dfCmpnySumry19Big5[~dfCmpnySumry19Big5['기업공개일'].isna()]['소속회사명'].nunique() /\
dfCmpnySumry19Big5['소속회사명'].nunique()

0.1670886075949367

In [279]:
dfCmpnySumry20Big5[~dfCmpnySumry20Big5['기업공개일'].isna()]['소속회사명'].nunique() /\
dfCmpnySumry20Big5['소속회사명'].nunique()

0.178117048346056

In [280]:
dfCmpnySumry21Big5[~dfCmpnySumry21Big5['기업공개일'].isna()]['소속회사명'].nunique() /\
dfCmpnySumry21Big5['소속회사명'].nunique()

0.1686746987951807

## 평균거래계열사수

In [281]:
len(dfTrans18Dmstc5Big) /\
dfCmpnySumry18Big5['소속회사명'].nunique()

11.20506329113924

In [282]:
len(dfTrans18Dmstc5Big[dfTrans18Dmstc5Big['매출사\n상/비'] == '상장']) /\
dfCmpnySumry18Big5[~dfCmpnySumry18Big5['기업공개일'].isna()]['소속회사명'].nunique()

25.742424242424242

In [283]:
len(dfTrans18Dmstc5Big[dfTrans18Dmstc5Big['매출사\n상/비'] == '비상장']) /\
dfCmpnySumry18Big5[dfCmpnySumry18Big5['기업공개일'].isna()]['소속회사명'].nunique()

8.288753799392097

In [284]:
len(dfTrans19Dmstc5Big) /\
dfCmpnySumry19Big5['소속회사명'].nunique()

12.521518987341771

In [285]:
len(dfTrans19Dmstc5Big[dfTrans19Dmstc5Big['매출사상/비'] == '상장']) /\
dfCmpnySumry19Big5[~dfCmpnySumry19Big5['기업공개일'].isna()]['소속회사명'].nunique()

30.075757575757574

In [286]:
len(dfTrans19Dmstc5Big[dfTrans19Dmstc5Big['매출사상/비'] == '비상장']) /\
dfCmpnySumry19Big5[dfCmpnySumry19Big5['기업공개일'].isna()]['소속회사명'].nunique()

9.0

In [287]:
len(dfTrans20Dmstc5Big) /\
dfCmpnySumry20Big5['소속회사명'].nunique()

11.755725190839694

In [288]:
len(dfTrans20Dmstc5Big[dfTrans20Dmstc5Big['매출사상/비'] == '상장']) /\
dfCmpnySumry20Big5[~dfCmpnySumry20Big5['기업공개일'].isna()]['소속회사명'].nunique()

26.32857142857143

In [289]:
len(dfTrans20Dmstc5Big[dfTrans20Dmstc5Big['매출사상/비'] == '비상장']) /\
dfCmpnySumry20Big5[dfCmpnySumry20Big5['기업공개일'].isna()]['소속회사명'].nunique()

8.597523219814242

In [290]:
len(dfTrans21Dmstc5Big) /\
dfCmpnySumry21Big5['소속회사명'].nunique()

11.086746987951807

In [291]:
len(dfTrans21Dmstc5Big[dfTrans21Dmstc5Big['매출사상/비'] == '상장']) /\
dfCmpnySumry21Big5[~dfCmpnySumry21Big5['기업공개일'].isna()]['소속회사명'].nunique()

26.857142857142858

In [292]:
len(dfTrans21Dmstc5Big[dfTrans21Dmstc5Big['매출사상/비'] == '비상장']) /\
dfCmpnySumry21Big5[dfCmpnySumry21Big5['기업공개일'].isna()]['소속회사명'].nunique()

7.808695652173913

## 평균내부거래금액

In [293]:
dfTrans18Dmstc5Big['매출액'].sum() /\
dfCmpnySumry18Big5['소속회사명'].nunique()

319931.6

In [294]:
dfTrans18Dmstc5Big[dfTrans18Dmstc5Big['매출사\n상/비'] == '상장']['매출액'].sum() /\
dfCmpnySumry18Big5[~dfCmpnySumry18Big5['기업공개일'].isna()]['소속회사명'].nunique()

867393.1818181818

In [295]:
dfTrans18Dmstc5Big[dfTrans18Dmstc5Big['매출사\n상/비'] == '비상장']['매출액'].sum() /\
dfCmpnySumry18Big5[dfCmpnySumry18Big5['기업공개일'].isna()]['소속회사명'].nunique()

210106.4802431611

In [296]:
dfTrans19Dmstc5Big['매출액'].sum() /\
dfCmpnySumry19Big5['소속회사명'].nunique()

342313.7974683544

In [297]:
dfTrans19Dmstc5Big[dfTrans19Dmstc5Big['매출사상/비'] == '상장']['매출액'].sum() /\
dfCmpnySumry19Big5[~dfCmpnySumry19Big5['기업공개일'].isna()]['소속회사명'].nunique()

944166.0

In [298]:
dfTrans19Dmstc5Big[dfTrans19Dmstc5Big['매출사상/비'] == '비상장']['매출액'].sum() /\
dfCmpnySumry19Big5[dfCmpnySumry19Big5['기업공개일'].isna()]['소속회사명'].nunique()

221577.48936170212

In [299]:
dfTrans20Dmstc5Big['매출액'].sum() /\
dfCmpnySumry20Big5['소속회사명'].nunique()

327637.3979643766

In [300]:
dfTrans20Dmstc5Big[dfTrans20Dmstc5Big['매출사상/비'] == '상장']['매출액'].sum() /\
dfCmpnySumry20Big5[~dfCmpnySumry20Big5['기업공개일'].isna()]['소속회사명'].nunique()

894019.5142857142

In [301]:
dfTrans20Dmstc5Big[dfTrans20Dmstc5Big['매출사상/비'] == '비상장']['매출액'].sum() /\
dfCmpnySumry20Big5[dfCmpnySumry20Big5['기업공개일'].isna()]['소속회사명'].nunique()

204892.04767801857

In [302]:
dfTrans21Dmstc5Big['매출액'].sum() /\
dfCmpnySumry21Big5['소속회사명'].nunique()

285201.70120481926

In [303]:
dfTrans21Dmstc5Big[dfTrans21Dmstc5Big['매출사상/비'] == '상장']['매출액'].sum() /\
dfCmpnySumry21Big5[~dfCmpnySumry21Big5['기업공개일'].isna()]['소속회사명'].nunique()

873724.0857142857

In [304]:
dfTrans21Dmstc5Big[dfTrans21Dmstc5Big['매출사상/비'] == '비상장']['매출액'].sum() /\
dfCmpnySumry21Big5[dfCmpnySumry21Big5['기업공개일'].isna()]['소속회사명'].nunique()

157933.99130434782

## 총매출액 대비 내부 거래 비중

In [305]:
dfTrans18Dmstc5Big[dfTrans18Dmstc5Big['매출사\n상/비'] == '상장']['매출액'].sum()

57247950.0

In [306]:
dfCmpnySumry18Big5['매출액'] = dfCmpnySumry18Big5['매출액'].replace(',', '', regex=True).astype(float)
dfCmpnySumry19Big5['매출액'] = dfCmpnySumry19Big5['매출액'].replace(',', '', regex=True).astype(float)
dfCmpnySumry20Big5['매출액'] = dfCmpnySumry20Big5['매출액'].replace(',', '', regex=True).astype(float)
dfCmpnySumry21Big5['매출액'] = dfCmpnySumry21Big5['매출액'].replace(',', '', regex=True).astype(float)

/var/folders/xw/1rpcyks10ps27v4w_vlbb6s00000gn/T/ipykernel_37869/3946549270.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfCmpnySumry18Big5['매출액'] = dfCmpnySumry18Big5['매출액'].replace(',', '', regex=True).astype(float)
/var/folders/xw/1rpcyks10ps27v4w_vlbb6s00000gn/T/ipykernel_37869/3946549270.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfCmpnySumry19Big5['매출액'] = dfCmpnySumry19Big5['매출액'].replace(',', '', regex=True).astype(float)
/var/folders/xw/1rpcyks10ps27v4w_vlbb6s00000gn/T/ipykernel_378

In [307]:
dfTrans18Dmstc5Big['매출액'].sum() /\
dfCmpnySumry18Big5['매출액'].sum()

0.14963495125599297

In [308]:
dfTrans18Dmstc5Big[dfTrans18Dmstc5Big['매출사\n상/비'] == '상장']['매출액'].sum() /\
dfCmpnySumry18Big5[~dfCmpnySumry18Big5['기업공개일'].isna()]['매출액'].sum()

0.09085207012251635

In [309]:
dfTrans18Dmstc5Big[dfTrans18Dmstc5Big['매출사\n상/비'] == '비상장']['매출액'].sum() /\
dfCmpnySumry18Big5[dfCmpnySumry18Big5['기업공개일'].isna()]['매출액'].sum()

0.3223826717899394

In [310]:
dfTrans19Dmstc5Big['매출액'].sum() /\
dfCmpnySumry19Big5['매출액'].sum()

0.1601033108277865

In [311]:
dfTrans19Dmstc5Big[dfTrans19Dmstc5Big['매출사상/비'] == '상장']['매출액'].sum() /\
dfCmpnySumry19Big5[~dfCmpnySumry19Big5['기업공개일'].isna()]['매출액'].sum()

0.09889337089264368

In [312]:
dfTrans19Dmstc5Big[dfTrans19Dmstc5Big['매출사상/비'] == '비상장']['매출액'].sum() /\
dfCmpnySumry19Big5[dfCmpnySumry19Big5['기업공개일'].isna()]['매출액'].sum()

0.33998353095183753

In [313]:
dfTrans20Dmstc5Big['매출액'].sum() /\
dfCmpnySumry20Big5['매출액'].sum()

0.15171804185428148

In [314]:
dfTrans20Dmstc5Big[dfTrans20Dmstc5Big['매출사상/비'] == '상장']['매출액'].sum() /\
dfCmpnySumry20Big5[~dfCmpnySumry20Big5['기업공개일'].isna()]['매출액'].sum()

0.09830974287965324

In [315]:
dfTrans20Dmstc5Big[dfTrans20Dmstc5Big['매출사상/비'] == '비상장']['매출액'].sum() /\
dfCmpnySumry20Big5[dfCmpnySumry20Big5['기업공개일'].isna()]['매출액'].sum()

0.31199966135516505

In [316]:
dfTrans21Dmstc5Big['매출액'].sum() /\
dfCmpnySumry21Big5['매출액'].sum() * 1000000

0.14176765782315714

In [317]:
dfTrans21Dmstc5Big[dfTrans21Dmstc5Big['매출사상/비'] == '상장']['매출액'].sum() /\
dfCmpnySumry21Big5[~dfCmpnySumry21Big5['기업공개일'].isna()]['매출액'].sum() * 1000000

0.09383785357345313

In [318]:
dfTrans21Dmstc5Big[dfTrans21Dmstc5Big['매출사상/비'] == '비상장']['매출액'].sum() /\
dfCmpnySumry21Big5[dfCmpnySumry21Big5['기업공개일'].isna()]['매출액'].sum() * 1000000

0.2975685061974676

# 직접지분율

In [334]:
pd.read_stata('../data/almeida/almeida_own1_2001-2018_0913수정.dta')

,firmid_before,firmid,fid,ultown,avpos,sd,steps,loop,lastcontrol,central,year,grpname,grpcode,grpname2,firmname
0,280.0,2.800000e+02,1.0,0.050860,2.147486,2.0,3.0,1.0,7.0,0.0,2000.0,lg,282.0,,
1,280.0,2.800000e+02,1.0,0.087554,2.283047,2.0,3.0,1.0,12.0,0.0,2001.0,lg,282.0,,
2,1402.0,1.402000e+03,1.0,0.000000,NaN,NaN,NaN,NaN,0.0,NaN,2012.0,kogas,24072.0,,
3,1402.0,1.402000e+03,1.0,0.000000,NaN,NaN,NaN,NaN,0.0,NaN,2013.0,kogas,24072.0,,
4,1402.0,1.402000e+03,1.0,0.000000,NaN,NaN,NaN,NaN,0.0,NaN,2014.0,kogas,24072.0,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22163,NaN,2.101743e+07,41.0,0.339856,2.999706,2.0,NaN,0.0,41.0,0.0,2016.0,ls,2402204.0,엘에스,베스트토요타
22164,NaN,2.303900e+07,42.0,0.239648,1.994576,1.0,NaN,0.0,25.0,0.0,2016.0,ls,2402204.0,엘에스,LS아이앤디
22165,NaN,1.801406e+09,43.0,0.132485,2.992250,2.0,NaN,0.0,25.0,0.0,2016.0,ls,2402204.0,엘에스,LS전선아시아
22166,NaN,1.802803e+09,44.0,0.339856,2.999706,2.0,NaN,0.0,41.0,0.0,2016.0,ls,2402204.0,엘에스,바이클로


In [345]:
dfTrans18Dmstc.head()

,year,year2,grpname,grpname2,grpcode,매출회사,매출회사id,매출사\n상/비,매입회사,매입회사id,매입사\n상/비,매출액,매출회사\n국내계열사계,매출회사\n해외계열사계,매출회사\n매출액총계,매출회사\n국내매출액,매출회사\n해외매출액,재벌순위,매출사 CFR\n(우선주포함),매출사 FAM\n(우선주포함)
2,2017,2018,DB,DB,365,DBInc.,352098,상장,DB금융투자,821640,상장,1424.0,43037.0,0,199530.0,123317.0,76213.0,36,0.401239,0.401239
3,2017,2018,DB,DB,365,DBInc.,352098,상장,DB라이텍,137656,상장,307.0,43037.0,0,199530.0,123317.0,76213.0,36,0.401239,0.401239
4,2017,2018,DB,DB,365,DBInc.,352098,상장,DB메탈,12097776,비상장,1566.0,43037.0,0,199530.0,123317.0,76213.0,36,0.401239,0.401239
5,2017,2018,DB,DB,365,DBInc.,352098,상장,DB생명보험,821420,비상장,5291.0,43037.0,0,199530.0,123317.0,76213.0,36,0.401239,0.401239
6,2017,2018,DB,DB,365,DBInc.,352098,상장,DB손해보험,870099,상장,13298.0,43037.0,0,199530.0,123317.0,76213.0,36,0.401239,0.401239


In [346]:
dfTrans18Dmstc[dfTrans18Dmstc['매출회사id'] == 196338]

,year,year2,grpname,grpname2,grpcode,매출회사,매출회사id,매출사\n상/비,매입회사,매입회사id,매입사\n상/비,매출액,매출회사\n국내계열사계,매출회사\n해외계열사계,매출회사\n매출액총계,매출회사\n국내매출액,매출회사\n해외매출액,재벌순위,매출사 CFR\n(우선주포함),매출사 FAM\n(우선주포함)
7940,2017,2018,nexon,넥슨,1902205,(유)와이즈키즈,196338,비상장,(유)엔엑스프로퍼티스,214529,비상장,22.0,26.0,0,63.0,55.0,8.0,45,1.0,1.0
7942,2017,2018,nexon,넥슨,1902205,(유)와이즈키즈,196338,비상장,네오플,198274,비상장,3.0,26.0,0,63.0,55.0,8.0,45,1.0,1.0
7959,2017,2018,nexon,넥슨,1902205,(유)와이즈키즈,196338,비상장,엔엑스씨,799173,비상장,1.0,26.0,0,63.0,55.0,8.0,45,1.0,1.0


# 직접지분율 실질지분율 데이터 병합

In [357]:
lInput18 = [pd.read_excel(fName, sheet_name=-1) for fName in glob.glob('../data/almeida/2018/in/*.xlsx')]
dfOwnInput18 = pd.concat(lInput18)
dfOwnInput18.head()

,grpname2,grpname,grpcode,firmname,firmid,own,own2,type,ownerid,ownername
0,넥슨,nexon,1902205.0,엔엑스씨,799173,0.982809,0.982809,0,NaN,NaN
1,넥슨,nexon,1902205.0,엔엑스씨,799173,0.017191,0.017191,1,196338.0,NaN
2,넥슨,nexon,1902205.0,엔엑스씨,799173,0.000000,0.000000,99,NaN,NaN
3,넥슨,nexon,1902205.0,(유)와이즈키즈,196338,1.000000,1.000000,0,NaN,NaN
4,넥슨,nexon,1902205.0,(유)와이즈키즈,196338,0.000000,0.000000,1,NaN,NaN


In [358]:
dfCmpnySumry18.columns

Index(['공개년월', '회계년', '기업집단명', '소속회사명', '대표자', '설립일', '계열편입일', '업종코드', '영위업종',
       '종업원수', '결산기일', '결산주총일', '기업공개일', '자산총액', '부채총액', '자본총액', '자본금', '매출액',
       '당기순이익', '구분', '법인등록번호', 'year2'],
      dtype='object')

In [361]:
dfCmpnySumry18.head()

,공개년월,회계년,기업집단명,소속회사명,대표자,설립일,계열편입일,업종코드,영위업종,종업원수,...,기업공개일,자산총액,부채총액,자본총액,자본금,매출액,당기순이익,구분,법인등록번호,year2
0,201805,2017,DB,(주)동부월드,최경진,1989/05/15,1997/06/02,R91,스포츠 및 오락관련 서비스업,76,...,NaN,"188,988","57,318","131,670","161,454",10894.0,"-3,672",회생절차,1101110628086,2018
1,201805,2017,DB,디비씨앤에스자동차손해사정(주),문수원,2011/03/17,2011/04/05,K66,금융 및 보험 관련 서비스업,742,...,NaN,"10,602","6,887","3,715",200,45777.0,"1,388",금융보험업,1101114558669,2018
2,201805,2017,DB,디비캐피탈(주),윤재인,1995/12/08,1995/12/08,K64,금융업,20,...,NaN,"186,265","103,745","82,520","62,014",11477.0,"2,344",금융보험업,1101111220641,2018
3,201805,2017,DB,디비금융투자(주),고원종,1982/12/20,1982/12/20,K66,금융 및 보험 관련 서비스업,968,...,1988/06/27,"5,401,797","4,808,879","592,918","212,232",806256.0,"2,591",금융보험업,1101110339956,2018
4,201805,2017,DB,디비생명보험(주),이태운,1989/04/14,1997/01/03,K65,보험 및 연금업,569,...,NaN,"10,663,512","10,024,347","639,165","193,331",2378149.0,"29,215",금융보험업,1101110623606,2018


In [363]:
apiKey = "7946dcde119af7656afc01157071c0ab9488b9ad"
myApi = api.Api(apiKey)
username = 'wonbok'
password = 'gkrtkrhk5034!'
dfCmpnySumry18 = myApi.scrapeKiscode(2018, username, password)

/Users/choidamian/Chaebol/notebooks/../scripts/api.py:379: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfTrgt.drop(columns=['rcmgCode'], inplace=True)


Rcmg code missing for 282 companies
Found kiscode for 1211110068750: 137656
Found kiscode for 1954110020532: H18678
Found kiscode for 1614110000352: 321141
Found kiscode for 1648110027701: 215559
Found kiscode for 1101116643541: IY8177
Found kiscode for 1411110039319: H77784
Found kiscode for 1350110081485: 116695
Found kiscode for 1101114306373: H78719
Found kiscode for 1101114177237: H35316
Found kiscode for 1101113936733: C85906
Found kiscode for 1101113207712: 377398
Found kiscode for 1101116172243: HT8917
Found kiscode for 1311110419472: HI0594
Found kiscode for 1311110484194: IQ8719
Found kiscode for 1101112405474: 715767
Found kiscode for 1101140141983: L56436
Found kiscode for 1311110382950: M01787
Found kiscode for 1101110522890: I59595
Found kiscode for 1101111242330: 887391
No company found for 1101116389757. Continuing...
Found kiscode for 2201110167693: IU7928
No company found for 1101011066683. Continuing...
Found kiscode for 1353110014555: G89184
Found kiscode for 110111

Found kiscode for 1344110025709: 375607
No company found for 1101115776059. Continuing...
Found kiscode for 1101116367365: IP7863
Found kiscode for 1101130016815: I67185
Found kiscode for 1101140177136: HO4630
Found kiscode for 1101116279809: IM3697
Found kiscode for 1411110002605: 362859
Found kiscode for 1101116091336: HR1377
Found kiscode for 1101115884146: HK4865
Found kiscode for 1450110013113: H82149
Found kiscode for 1101110825624: 635049
Found kiscode for 1101110270895: 351113
Found kiscode for 2043110008603: 133874
Found kiscode for 1101115604700: CC2041
Found kiscode for 1313110079553: C06931
Found kiscode for 1412110051816: F15967
Found kiscode for 1101110017867: 340120
Found kiscode for 1350110106150: 158650
Found kiscode for 1101110004682: 382663
Found kiscode for 1342110183642: IM6527
Found kiscode for 1601110123185: 196174
Found kiscode for 1942110046004: 174719
Found kiscode for 1801111066538: IM6418
Found kiscode for 1101112915605: 227533
Found kiscode for 124411004334

In [367]:
dfCmpnySumry18.loc[dfCmpnySumry18[dfCmpnySumry18['entrprsNm'] == '(주)락스퀘어'].index, 'rcmgCode'] = 2830000000
dfCmpnySumry18.loc[dfCmpnySumry18[dfCmpnySumry18['entrprsNm'] == '(주)엔엑스씨'].index, 'rcmgCode'] = 799173
dfCmpnySumry18.loc[dfCmpnySumry18[dfCmpnySumry18['entrprsNm'] == '(주)오투리조트'].index, 'rcmgCode'] = 548906
dfCmpnySumry18.loc[dfCmpnySumry18[dfCmpnySumry18['entrprsNm'] == '유진에스비홀딩스(주)'].index, 'rcmgCode'] = 1902909737
dfCmpnySumry18.loc[dfCmpnySumry18[dfCmpnySumry18['entrprsNm'] == '(주)에이플러스비'].index, 'rcmgCode'] = 2830000000
dfCmpnySumry18.loc[dfCmpnySumry18[dfCmpnySumry18['entrprsNm'] == '(주)플라잉모터스'].index, 'rcmgCode'] = 2301508963

In [368]:
dfCmpnySumry18[dfCmpnySumry18['rcmgCode'].isna()]

,publicYm,fisclYr,unityGrupNm,entrprsNm,rprsntvNm,fondDe,grinil,indutyCode,indutyNm,ordtmEmplyCo,...,assetsTotamt,debtTotamt,caplTotamt,caplAmount,selngAmount,thstrmNtpfAmount,reconstrctur,jurirno,year2,rcmgCode


In [369]:
dfCmpnySumry18.to_excel('../data/cmpnySumry/cmpnySumry2018Eng.xlsx', index=False)

In [370]:
dfCmpnySumry18C = dfCmpnySumry18[dfCmpnySumry18['unityGrupNm'].isin(dfAffilNum18['chaebol'])]
dfCmpnySumry18C['unityGrupNm'].nunique()

52

In [376]:
dfCmpnySumry18C['firmid'] = dfCmpnySumry18C['rcmgCode']

/var/folders/xw/1rpcyks10ps27v4w_vlbb6s00000gn/T/ipykernel_37869/1120564480.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfCmpnySumry18C['firmid'] = dfCmpnySumry18C['rcmgCode']


In [372]:
dfOwnInput18.head()

,grpname2,grpname,grpcode,firmname,firmid,own,own2,type,ownerid,ownername
0,넥슨,nexon,1902205.0,엔엑스씨,799173,0.982809,0.982809,0,NaN,NaN
1,넥슨,nexon,1902205.0,엔엑스씨,799173,0.017191,0.017191,1,196338.0,NaN
2,넥슨,nexon,1902205.0,엔엑스씨,799173,0.000000,0.000000,99,NaN,NaN
3,넥슨,nexon,1902205.0,(유)와이즈키즈,196338,1.000000,1.000000,0,NaN,NaN
4,넥슨,nexon,1902205.0,(유)와이즈키즈,196338,0.000000,0.000000,1,NaN,NaN


In [377]:
dfOwnInput18['직접지분율'] = dfOwnInput18['own']

In [378]:
dfOwnOutput18 = pd.read_stata("../data/almeida/almeida_own1_2001-2018_0913수정.dta")
dfOwnOutput18 = dfOwnOutput18[dfOwnOutput18['year'] == 2017]
dfOwnOutput18['실질지분율'] = dfOwnOutput18['ultown']
dfOwnOutput18.head()

,firmid_before,firmid,fid,ultown,avpos,sd,steps,loop,lastcontrol,central,year,grpname,grpcode,grpname2,firmname,실질지분율
62,5086.0,5086.0,1.0,0.003523,4.176080,3.0,NaN,0.0,8.0,0.246155,2017.0,eugenes,653.0,유진,케이비에프,0.003523
76,6687.0,6687.0,1.0,0.058183,1.991406,1.0,NaN,0.0,15.0,0.000000,2017.0,hyosung,971.0,효성,신화인터텍,0.058183
78,NaN,7080.0,1.0,1.000000,1.000000,1.0,NaN,0.0,100.0,0.000000,2017.0,kcc,184.0,케이씨씨,대호포장,1.000000
99,7131.0,7131.0,1.0,0.511612,1.000000,1.0,NaN,0.0,51.0,0.000000,2017.0,booyoung,16032.0,부영,광영토건,0.511612
116,7158.0,7158.0,1.0,0.475000,1.694737,1.0,NaN,0.0,47.0,6.590908,2017.0,youngpoong,632.0,영풍,영풍문고,0.475000


In [379]:
dfCmpnySumry18C = dfCmpnySumry18C.merge(dfOwnInput18[dfOwnInput18['type'] == 0][['firmid', '직접지분율']], on='firmid', how='left')\
                                 .merge(dfOwnOutput18[['firmid', '실질지분율']], on='firmid', how='left')
dfCmpnySumry18C.head()

,publicYm,fisclYr,unityGrupNm,entrprsNm,rprsntvNm,fondDe,grinil,indutyCode,indutyNm,ordtmEmplyCo,...,caplAmount,selngAmount,thstrmNtpfAmount,reconstrctur,jurirno,year2,rcmgCode,firmid,직접지분율,실질지분율
0,201805,2017,DB,(주)동부월드,최경진,1989/05/15,1997/06/02,R91,스포츠 및 오락관련 서비스업,76,...,"161,454","10,894","-3,672",회생절차,1101110628086,2018,910813.0,910813.0,0.000000,0.099492
1,201805,2017,DB,디비씨앤에스자동차손해사정(주),문수원,2011/03/17,2011/04/05,K66,금융 및 보험 관련 서비스업,742,...,200,"45,777","1,388",금융보험업,1101114558669,2018,18004172.0,18004172.0,0.000000,0.181040
2,201805,2017,DB,디비캐피탈(주),윤재인,1995/12/08,1995/12/08,K64,금융업,20,...,"62,014","11,477","2,344",금융보험업,1101111220641,2018,56018.0,56018.0,0.000000,0.166868
3,201805,2017,DB,디비금융투자(주),고원종,1982/12/20,1982/12/20,K66,금융 및 보험 관련 서비스업,968,...,"212,232","806,256","2,591",금융보험업,1101110339956,2018,821640.0,821640.0,0.129058,0.174456
4,201805,2017,DB,디비생명보험(주),이태운,1989/04/14,1997/01/03,K65,보험 및 연금업,569,...,"193,331","2,378,149","29,215",금융보험업,1101110623606,2018,821420.0,821420.0,0.001217,0.141807


In [385]:
dfOwnInput18['매출회사id'] = dfOwnInput18['firmid']
dfOwnOutput18['매출회사id'] = dfOwnOutput18['firmid']
dfOwnOutput18['매입회사id'] = dfOwnOutput18['firmid']

In [382]:
dfOwnOutput18['매출회사 실질지분율'] = dfOwnOutput18['실질지분율']
dfOwnOutput18['매입회사 실질지분율'] = dfOwnOutput18['실질지분율']

In [381]:
dfTrans18Dmstc.head()

,year,year2,grpname,grpname2,grpcode,매출회사,매출회사id,매출사\n상/비,매입회사,매입회사id,매입사\n상/비,매출액,매출회사\n국내계열사계,매출회사\n해외계열사계,매출회사\n매출액총계,매출회사\n국내매출액,매출회사\n해외매출액,재벌순위,매출사 CFR\n(우선주포함),매출사 FAM\n(우선주포함)
2,2017,2018,DB,DB,365,DBInc.,352098,상장,DB금융투자,821640,상장,1424.0,43037.0,0,199530.0,123317.0,76213.0,36,0.401239,0.401239
3,2017,2018,DB,DB,365,DBInc.,352098,상장,DB라이텍,137656,상장,307.0,43037.0,0,199530.0,123317.0,76213.0,36,0.401239,0.401239
4,2017,2018,DB,DB,365,DBInc.,352098,상장,DB메탈,12097776,비상장,1566.0,43037.0,0,199530.0,123317.0,76213.0,36,0.401239,0.401239
5,2017,2018,DB,DB,365,DBInc.,352098,상장,DB생명보험,821420,비상장,5291.0,43037.0,0,199530.0,123317.0,76213.0,36,0.401239,0.401239
6,2017,2018,DB,DB,365,DBInc.,352098,상장,DB손해보험,870099,상장,13298.0,43037.0,0,199530.0,123317.0,76213.0,36,0.401239,0.401239


In [386]:
dfTrans18Dmstc = dfTrans18Dmstc.merge(dfOwnInput18[dfOwnInput18['type'] == 0][['매출회사id', '직접지분율']], on='매출회사id', how='left')\
                               .merge(dfOwnOutput18[['매출회사id', '매출회사 실질지분율']], on='매출회사id', how='left')\
                               .merge(dfOwnOutput18[['매입회사id', '매입회사 실질지분율']], on='매입회사id', how='left')

In [387]:
dfTrans18Dmstc.head()

,year,year2,grpname,grpname2,grpcode,매출회사,매출회사id,매출사\n상/비,매입회사,매입회사id,...,매출회사\n해외계열사계,매출회사\n매출액총계,매출회사\n국내매출액,매출회사\n해외매출액,재벌순위,매출사 CFR\n(우선주포함),매출사 FAM\n(우선주포함),직접지분율,매출회사 실질지분율,매입회사 실질지분율
0,2017,2018,DB,DB,365,DBInc.,352098,상장,DB금융투자,821640,...,0,199530.0,123317.0,76213.0,36,0.401239,0.401239,0.401239,0.401239,0.174456
1,2017,2018,DB,DB,365,DBInc.,352098,상장,DB라이텍,137656,...,0,199530.0,123317.0,76213.0,36,0.401239,0.401239,0.401239,0.401239,0.060650
2,2017,2018,DB,DB,365,DBInc.,352098,상장,DB메탈,12097776,...,0,199530.0,123317.0,76213.0,36,0.401239,0.401239,0.401239,0.401239,0.589516
3,2017,2018,DB,DB,365,DBInc.,352098,상장,DB생명보험,821420,...,0,199530.0,123317.0,76213.0,36,0.401239,0.401239,0.401239,0.401239,0.141807
4,2017,2018,DB,DB,365,DBInc.,352098,상장,DB손해보험,870099,...,0,199530.0,123317.0,76213.0,36,0.401239,0.401239,0.401239,0.401239,0.181040


In [403]:
lInput19 = [pd.read_excel(fName, sheet_name=-1) for fName in glob.glob('../data/almeida/2019/in/*.xlsx')]
dfOwnInput19 = pd.concat(lInput19)
dfOwnInput19.head()

,grpname2,grpname,grpcode,금융비금융,firmname,firmid,type,own,own2,ownername,ownerid,year2,소속회사명,shares
0,엘지,lg,282.0,비금융회사,엘지,350079.0,0.0,0.319065,0.343866,NaN,NaN,NaN,NaN,NaN
1,엘지,lg,282.0,비금융회사,엘지,350079.0,1.0,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN
2,엘지,lg,282.0,비금융회사,엘지,350079.0,99.0,0.656134,0.656134,NaN,NaN,NaN,NaN,NaN
3,엘지,lg,282.0,비금융회사,엘지화학,162209.0,0.0,0.000000,0.021810,NaN,NaN,NaN,NaN,NaN
4,엘지,lg,282.0,비금융회사,엘지화학,162209.0,1.0,0.300637,0.300637,엘지,350079.0,NaN,NaN,NaN


In [402]:
dfOwnInput19.columns

Index(['grpname2', 'grpname', 'grpcode', '금융비금융', 'firmname', 'firmid', 'type',
       'own', 'own2', 'ownername', 'ownerid', 'year2', '소속회사명', 'shares'],
      dtype='object')

In [404]:
dfOwnInput19[~dfOwnInput19['shares'].isna()]

,grpname2,grpname,grpcode,금융비금융,firmname,firmid,type,own,own2,ownername,ownerid,year2,소속회사명,shares
1,한진,hanjin,926.0,비금융회사,대한항공,990027.0,1.0,0.296249,0.296249,한진칼,21083149.0,2019.0,NaN,0.296249
2,한진,hanjin,926.0,비금융회사,대한항공,990027.0,99.0,0.669899,0.669899,NaN,NaN,2019.0,NaN,0.669899
3,한진,hanjin,926.0,비금융회사,한진칼,21083149.0,0.0,0.253441,0.287158,NaN,NaN,2019.0,NaN,0.023201
5,한진,hanjin,926.0,비금융회사,한진칼,21083149.0,99.0,0.712842,0.712842,NaN,NaN,2019.0,NaN,0.712842
6,한진,hanjin,926.0,비금융회사,한진,710105.0,0.0,0.069708,0.123683,NaN,NaN,2019.0,NaN,0.000334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,한진,hanjin,926.0,비금융회사,더블유에이씨항공서비스,32033876.0,1.0,1.000000,1.000000,세계혼재항공화물,730025.0,2019.0,NaN,1.000000
93,한진,hanjin,926.0,비금융회사,더블유에이씨항공서비스,32033876.0,99.0,0.000000,0.000000,NaN,NaN,2019.0,NaN,1.000000
94,한진,hanjin,926.0,비금융회사,서화무역,691992.0,0.0,0.735294,0.735294,NaN,NaN,2019.0,NaN,0.000000
95,한진,hanjin,926.0,비금융회사,서화무역,691992.0,1.0,0.000000,0.000000,NaN,NaN,2019.0,NaN,0.000000


In [424]:
dfCmpnySumry19 = pd.read_excel('../data/cmpnySumry/cmpnySumry2019Eng.xlsx')
dfCmpnySumry19 = dfCmpnySumry19[dfCmpnySumry19['unityGrupNm'].isin(dfAffilNum19['chaebol'])]
dfCmpnySumry19.loc[dfCmpnySumry19[dfCmpnySumry19['rcmgCode'].isna()].index, 'rcmgCode'] = 2830000000
dfCmpnySumry19['unityGrupNm'].nunique()

51

In [425]:
dfOwnOutput19 = pd.read_stata('../data/almeida/2019/out/almeida_own1_2019.dta')
dfOwnOutput19.head()

,year,year2,grpname2,grpname,grpcode,firmname,firmid,fid,ultown,avpos,sd,steps,loop,lastcontrol,central
0,2018.0,2019.0,애경,aekyung,615,우영운수,47919,1.0,1.000000,1.000000,1.0,NaN,0.0,100.0,0.000000
1,2018.0,2019.0,애경,aekyung,615,수원애경역사,73220,2.0,0.605655,3.206202,2.0,NaN,0.0,64.0,0.000000
2,2018.0,2019.0,애경,aekyung,615,애드미션,95760,3.0,0.852500,1.000000,1.0,NaN,0.0,85.0,0.000000
3,2018.0,2019.0,애경,aekyung,615,에이케이레저,99486,4.0,1.000000,2.569844,2.0,NaN,0.0,100.0,0.000000
4,2018.0,2019.0,애경,aekyung,615,에이케이에스앤디,125359,5.0,0.719310,2.206202,1.0,NaN,0.0,64.0,6.736839


In [426]:
dfCmpnySumry19['firmid'] = dfCmpnySumry19['rcmgCode']
dfOwnInput19['직접지분율'] = dfOwnInput19['own']
dfOwnInput19['매출회사id'] = dfOwnInput19['firmid']
dfOwnOutput19['매출회사id'] = dfOwnOutput19['firmid']
dfOwnOutput19['매입회사id'] = dfOwnOutput19['firmid']
dfOwnOutput19['매출회사 실질지분율'] = dfOwnOutput19['ultown']
dfOwnOutput19['매입회사 실질지분율'] = dfOwnOutput19['ultown']
dfOwnOutput19['실질지분율'] = dfOwnOutput19['ultown']

In [428]:
dfCmpnySumry19['firmid'] = dfCmpnySumry19['firmid'].astype('int32')

In [441]:
dfOwnInput19 = dfOwnInput19[~dfOwnInput19['firmid'].isna()]

In [447]:
dfOwnInput19['firmid'] = dfOwnInput19['firmid'].astype('int32')

/var/folders/xw/1rpcyks10ps27v4w_vlbb6s00000gn/T/ipykernel_37869/534835833.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfOwnInput19['firmid'] = dfOwnInput19['firmid'].astype('int32')


In [443]:
dfCmpnySumry19 = dfCmpnySumry19.merge(dfOwnInput19[dfOwnInput19['type'] == 0][['firmid', '직접지분율']], on='firmid', how='left')\
                               .merge(dfOwnOutput19[['firmid', '실질지분율']], on='firmid', how='left')
dfCmpnySumry19.head()

,publicYm,fisclYr,unityGrupNm,entrprsNm,rprsntvNm,fondDe,grinil,indutyCode,indutyNm,ordtmEmplyCo,...,caplAmount,selngAmount,thstrmNtpfAmount,reconstrctur,jurirno,year2,rcmgCode,firmid,직접지분율,실질지분율
0,201905,2018,DB,디비생명보험(주),이태운,1989/04/14,1997/01/03,K65,보험 및 연금업,563,...,"193,331","2,180,825","24,637",금융보험업,1101110623606,2019,821420.0,821420,0.001217,0.141807
1,201905,2018,DB,디비금융서비스(주),황원기,2014/01/13,2014/03/01,K66,금융 및 보험관련 서비스업,30,...,"14,001","13,311","-2,499",금융보험업,1101115314474,2019,23044292.0,23044292,0.000000,0.181040
2,201905,2018,DB,디비에프아이에스(주),강운식,2014/12/01,2018/06/01,J62,"컴퓨터 프로그래밍, 시스템 통합 및 관리업",217,...,"1,000","47,269","12,063",일반회사,1101115576727,2019,24038019.0,24038019,0.000000,0.394799
3,201905,2018,DB,디비금융투자(주),고원종,1982/12/20,1982/12/20,K66,금융 및 보험관련 서비스업,892,...,"212,232","725,156","46,854",금융보험업,1101110339956,2019,821640.0,821640,0.125264,0.170662
4,201905,2018,DB,디비인베스트(주),최경진,2009/11/16,2010/01/04,M71,전문 서비스업,1,...,"306,500",933,-267,일반회사,1101114220359,2019,16003715.0,16003715,1.000000,1.000000


In [451]:
dfTrans19Dmstc = dfTrans19Dmstc.merge(dfOwnInput19[dfOwnInput19['type'] == 0][['매출회사id', '직접지분율']], on='매출회사id', how='left')\
                               .merge(dfOwnOutput19[['매출회사id', '매출회사 실질지분율']], on='매출회사id', how='left')\
                               .merge(dfOwnOutput19[['매입회사id', '매입회사 실질지분율']], on='매입회사id', how='left')

In [452]:
dfTrans19Dmstc.head()

,매출회사,매입회사,매출액,국내계열사계(매출액),해외계열사계(매출액),국내매출액,해외매출액,매출액총계,year2,year,...,매출액Val,매입사상/비,매입사금융/비금융,grpname,grpcode,매출회사id,매입회사id,직접지분율,매출회사 실질지분율,매입회사 실질지분율
0,삼성전자,삼성물산,78676.0,2269950.0,149436938.0,16821275.0,153560595.0,170381870.0,2019,2018,...,170381870.0,상장,비금융,samsung,511,380725,810053,0.051111,0.08541,0.311626
1,삼성전자,삼성바이오에피스,1.0,2269950.0,149436938.0,16821275.0,153560595.0,170381870.0,2019,2018,...,170381870.0,비상장,비금융,samsung,511,380725,18082125,0.051111,0.08541,0.081307
2,삼성전자,삼성디스플레이,164404.0,2269950.0,149436938.0,16821275.0,153560595.0,170381870.0,2019,2018,...,170381870.0,비상장,비금융,samsung,511,380725,18082126,0.051111,0.08541,0.074901
3,삼성전자,세메스,5.0,2269950.0,149436938.0,16821275.0,153560595.0,170381870.0,2019,2018,...,170381870.0,비상장,비금융,samsung,511,380725,314528,0.051111,0.08541,0.078187
4,삼성전자,삼성전자서비스,456109.0,2269950.0,149436938.0,16821275.0,153560595.0,170381870.0,2019,2018,...,170381870.0,비상장,비금융,samsung,511,380725,952133,0.051111,0.08541,0.084836


In [320]:
dfTrans19Dmstc.head()

,매출회사,매입회사,매출액,국내계열사계(매출액),해외계열사계(매출액),국내매출액,해외매출액,매출액총계,year2,year,grpname2,매출사상/비,매출사금융/비금융,매출액Val,매입사상/비,매입사금융/비금융,grpname,grpcode,매출회사id,매입회사id
0,삼성전자,삼성물산,78676.0,2269950.0,149436938.0,16821275.0,153560595.0,170381870.0,2019,2018,삼성,상장,비금융,170381870.0,상장,비금융,samsung,511,380725,810053
1,삼성전자,삼성바이오에피스,1.0,2269950.0,149436938.0,16821275.0,153560595.0,170381870.0,2019,2018,삼성,상장,비금융,170381870.0,비상장,비금융,samsung,511,380725,18082125
2,삼성전자,삼성디스플레이,164404.0,2269950.0,149436938.0,16821275.0,153560595.0,170381870.0,2019,2018,삼성,상장,비금융,170381870.0,비상장,비금융,samsung,511,380725,18082126
3,삼성전자,세메스,5.0,2269950.0,149436938.0,16821275.0,153560595.0,170381870.0,2019,2018,삼성,상장,비금융,170381870.0,비상장,비금융,samsung,511,380725,314528
4,삼성전자,삼성전자서비스,456109.0,2269950.0,149436938.0,16821275.0,153560595.0,170381870.0,2019,2018,삼성,상장,비금융,170381870.0,비상장,비금융,samsung,511,380725,952133


In [464]:
lInput20 = [pd.read_excel(fName, sheet_name=-1).rename(columns={'금융비금융 ':'금융비금융', 'firmid ':'firmid', 'firmname ': 'firmname', 'type ': 'type'})\
            for fName in glob.glob('../data/almeida/2020/in/*.xlsx')]
dfOwnInput20 = pd.concat(lInput20)
dfOwnInput20.head()

,금융비금융,grpname2,grpname,grpcode,firmname,firmid,type,own,own2,ownername,ownerid,소속회사명
0,비금융회사,금호석유화학,kpc,11140.0,금호석유화학,351350.0,0.0,0.225941,0.393303,NaN,NaN,NaN
1,비금융회사,금호석유화학,kpc,11140.0,금호석유화학,351350.0,99.0,0.606697,0.606697,NaN,NaN,NaN
2,비금융회사,금호석유화학,kpc,11140.0,금호피앤비화학,145509.0,1.0,1.000000,1.000000,금호석유화학,351350.0,NaN
3,비금융회사,금호석유화학,kpc,11140.0,금호미쓰이화학,356468.0,1.0,0.500000,0.500000,금호석유화학,351350.0,NaN
4,비금융회사,금호석유화학,kpc,11140.0,금호미쓰이화학,356468.0,99.0,0.500000,0.500000,NaN,NaN,NaN


In [481]:
dfCmpnySumry20 = pd.read_excel('../data/cmpnySumry/cmpnySumry2020Eng.xlsx')
dfCmpnySumry20 = dfCmpnySumry20[dfCmpnySumry20['unityGrupNm'].isin(dfAffilNum20['chaebol'])]
dfCmpnySumry20.loc[dfCmpnySumry20[dfCmpnySumry20['rcmgCode'].isna()].index, 'rcmgCode'] = 2830000000
dfCmpnySumry20['unityGrupNm'].nunique()

55

In [470]:
dfOwnOutput20 = pd.read_stata('../data/almeida/2020/out/almeida_own1_2020.dta')
dfOwnOutput20.head()

,grpname2,grpname,grpcode,firmname,firmid,fid,ultown,avpos,sd,steps,loop,lastcontrol,central,year2,year
0,삼성,samsung,511,스테코,39224.0,1.0,0.059979,2.463976,2.0,NaN,0.0,16.0,0.0,2020.0,2019.0
1,삼성,samsung,511,삼성전자판매,102357.0,2.0,0.085685,2.463976,2.0,NaN,0.0,16.0,0.0,2020.0,2019.0
2,삼성,samsung,511,삼성벤처투자,122088.0,3.0,0.089795,2.410684,2.0,NaN,0.0,29.0,0.0,2020.0,2019.0
3,삼성,samsung,511,삼성전자로지텍,125346.0,4.0,0.085685,2.463976,2.0,NaN,0.0,16.0,0.0,2020.0,2019.0
4,삼성,samsung,511,시큐아이,131940.0,5.0,0.165134,2.281845,2.0,NaN,0.0,31.0,0.0,2020.0,2019.0


In [483]:
dfCmpnySumry20['firmid'] = dfCmpnySumry20['rcmgCode']
dfOwnInput20['직접지분율'] = dfOwnInput20['own']
dfOwnInput20['매출회사id'] = dfOwnInput20['firmid']
dfOwnOutput20['매출회사id'] = dfOwnOutput20['firmid']
dfOwnOutput20['매입회사id'] = dfOwnOutput20['firmid']
dfOwnOutput20['매출회사 실질지분율'] = dfOwnOutput20['ultown']
dfOwnOutput20['매입회사 실질지분율'] = dfOwnOutput20['ultown']
dfOwnOutput20['실질지분율'] = dfOwnOutput20['ultown']

In [477]:
dfCmpnySumry19.columns

Index(['publicYm', 'fisclYr', 'unityGrupNm', 'entrprsNm', 'rprsntvNm',
       'fondDe', 'grinil', 'indutyCode', 'indutyNm', 'ordtmEmplyCo',
       'stacntDudt', 'beforeBsnsStacntDe', 'entrprsOthbcDe', 'assetsTotamt',
       'debtTotamt', 'caplTotamt', 'caplAmount', 'selngAmount',
       'thstrmNtpfAmount', 'reconstrctur', 'jurirno', 'year2', 'rcmgCode',
       'firmid', '직접지분율_x', '실질지분율_x'],
      dtype='object')

In [484]:
dfCmpnySumry20[dfCmpnySumry20['firmid'].isna()]

,publicYm,fisclYr,unityGrupNm,entrprsNm,rprsntvNm,fondDe,grinil,indutyCode,indutyNm,ordtmEmplyCo,...,debtTotamt,caplTotamt,caplAmount,selngAmount,thstrmNtpfAmount,reconstrctur,jurirno,year2,rcmgCode,firmid


In [485]:
dfCmpnySumry20['firmid'] = dfCmpnySumry20['firmid'].astype('int')

In [486]:
dfOwnInput20['firmid'] = dfOwnInput20['firmid'].astype('int')

In [487]:
dfOwnOutput20['firmid'] = dfOwnOutput20['firmid'].astype('int')

In [488]:
dfCmpnySumry20 = dfCmpnySumry20.merge(dfOwnInput20[dfOwnInput20['type'] == 0][['firmid', '직접지분율']], on='firmid', how='left')\
                               .merge(dfOwnOutput20[['firmid', '실질지분율']], on='firmid', how='left')
dfCmpnySumry20.head()

,publicYm,fisclYr,unityGrupNm,entrprsNm,rprsntvNm,fondDe,grinil,indutyCode,indutyNm,ordtmEmplyCo,...,caplAmount,selngAmount,thstrmNtpfAmount,reconstrctur,jurirno,year2,rcmgCode,firmid,직접지분율,실질지분율
0,202005,2019,넥슨,(유)가승개발,서춘식,2016/01/21,2016/01/21,R91,스포츠 및 오락관련 서비스업,3,...,100,0,-465,일반회사,1345140003989,2020,1.802408e+09,1802408099,NaN,0.500000
1,202005,2019,넥슨,(유)가승개발,서춘식,2016/01/21,2016/01/21,R91,스포츠 및 오락관련 서비스업,3,...,100,0,-465,일반회사,1345140003989,2020,1.802408e+09,1802408099,NaN,0.500000
2,202005,2019,지에스,(유)가승개발,서춘식,2016/01/21,2016/04/01,R91,스포츠 및 오락관련 서비스업,3,...,100,NaN,-465,일반회사,1345140003989,2020,1.802408e+09,1802408099,NaN,0.500000
3,202005,2019,지에스,(유)가승개발,서춘식,2016/01/21,2016/04/01,R91,스포츠 및 오락관련 서비스업,3,...,100,NaN,-465,일반회사,1345140003989,2020,1.802408e+09,1802408099,NaN,0.500000
4,202005,2019,카카오,(유)매디슨카운티의다리,이명한,2016/11/16,2020/02/03,R90,"창작, 예술 및 여가관련 서비스업",0,...,10,3,2,일반회사,1101140192605,2020,1.803409e+09,1803408949,0.0,0.061863


In [489]:
dfTrans20Dmstc = dfTrans20Dmstc.merge(dfOwnInput20[dfOwnInput20['type'] == 0][['매출회사id', '직접지분율']], on='매출회사id', how='left')\
                               .merge(dfOwnOutput20[['매출회사id', '매출회사 실질지분율']], on='매출회사id', how='left')\
                               .merge(dfOwnOutput20[['매입회사id', '매입회사 실질지분율']], on='매입회사id', how='left')

In [490]:
dfTrans20Dmstc.head()

,매출회사,매입회사,매출액,국내계열사계(매출액),해외계열사계(매출액),국내매출액,해외매출액,매출액총계,year2,year,...,매출액Val,매입사상/비,매입사금융/비금융,grpname,grpcode,매출회사id,매입회사id,직접지분율,매출회사 실질지분율,매입회사 실질지분율
0,삼성전자,삼성물산,110238.0,2422149.0,130563681.0,20300939.0,134471920.0,154772859.0,2020,2019,...,154772859.0,상장,비금융,samsung,511,380725,810053,0.051113,0.085685,0.316260
1,삼성전자,삼성바이오에피스,4.0,2422149.0,130563681.0,20300939.0,134471920.0,154772859.0,2020,2019,...,154772859.0,비상장,비금융,samsung,511,380725,18082125,0.051113,0.085685,0.082311
2,삼성전자,삼성디스플레이,279363.0,2422149.0,130563681.0,20300939.0,134471920.0,154772859.0,2020,2019,...,154772859.0,비상장,비금융,samsung,511,380725,18082126,0.051113,0.085685,0.075149
3,삼성전자,세메스,7.0,2422149.0,130563681.0,20300939.0,134471920.0,154772859.0,2020,2019,...,154772859.0,비상장,비금융,samsung,511,380725,314528,0.051113,0.085685,0.078438
4,삼성전자,삼성전자서비스,453084.0,2422149.0,130563681.0,20300939.0,134471920.0,154772859.0,2020,2019,...,154772859.0,비상장,비금융,samsung,511,380725,952133,0.051113,0.085685,0.085109


In [491]:
lInput21 = [pd.read_excel(fName) for fName in glob.glob('../data/almeida/2021/in/*.xlsx')]
dfOwnInput21 = pd.concat(lInput21)
dfOwnInput21.head()

,금융비금융,grpname2,grpname,grpcode,firmname,firmid,type,own,own2,ownername,ownerid
0,비금융회사,영풍,youngpoong,632,고려아연,370380,0,0.053914,0.184120,NaN,NaN
1,비금융회사,영풍,youngpoong,632,고려아연,370380,1,0.274870,0.274870,영풍,370274.0
2,비금융회사,영풍,youngpoong,632,고려아연,370380,1,0.015639,0.015639,영풍정밀,388947.0
3,비금융회사,영풍,youngpoong,632,고려아연,370380,1,0.009889,0.009889,유미개발,28088.0
4,비금융회사,영풍,youngpoong,632,고려아연,370380,1,0.001125,0.001125,테라닉스,625507.0


In [575]:
dfCmpnySumry21 = pd.read_excel('../data/cmpnySumry/cmpnySumry2021Eng.xlsx')
dfCmpnySumry21 = dfCmpnySumry21[dfCmpnySumry21['unityGrupNm'].isin(dfAffilNum21['chaebol'])]
dfCmpnySumry21['unityGrupNm'].nunique()

60

In [552]:
dfCmpnySumry21[dfCmpnySumry21['rcmgCode'].isna()]

,publicYm,fisclYr,unityGrupNm,entrprsNm,rprsntvNm,fondDe,grinil,indutyCode,indutyNm,ordtmEmplyCo,...,assetsTotamt,debtTotamt,caplTotamt,caplAmount,selngAmount,thstrmNtpfAmount,reconstrctur,jurirno,year2,rcmgCode


In [495]:
dfOwnOutput21 = pd.read_excel('../data/almeida/2021/out/almeida_own1_2021.xlsx')
dfOwnOutput21.head()

,year,year2,grpname2,grpname,grpcode,firmname,firmid,fid,ultown,avpos,sd,steps,loop,lastcontrol,central
0,2020,2021,유진,eugenes,653,케이비에프,5086,1,0.003715,4.142763,3.0,NaN,0.0,8,0.163265
1,2020,2021,유진,eugenes,653,케이비에프,5086,1,0.003715,4.142763,3.0,NaN,0.0,8,0.163265
2,2020,2021,유진,eugenes,653,케이비에프,5086,1,0.003715,4.142763,3.0,NaN,0.0,8,0.163265
3,2020,2021,유진,eugenes,653,한성레미콘,102640,2,0.103584,3.275612,3.0,NaN,0.0,29,0.755102
4,2020,2021,유진,eugenes,653,한성레미콘,102640,2,0.103584,3.275612,3.0,NaN,0.0,29,0.755102


In [576]:
dfCmpnySumry21['firmid'] = dfCmpnySumry21['rcmgCode']
dfOwnInput21['직접지분율'] = dfOwnInput21['own']
dfOwnInput21['매출회사id'] = dfOwnInput21['firmid']
dfOwnOutput21['매출회사id'] = dfOwnOutput21['firmid']
dfOwnOutput21['매입회사id'] = dfOwnOutput21['firmid']
dfOwnOutput21['매출회사 실질지분율'] = dfOwnOutput21['ultown']
dfOwnOutput21['매입회사 실질지분율'] = dfOwnOutput21['ultown']
dfOwnOutput21['실질지분율'] = dfOwnOutput21['ultown']

/var/folders/xw/1rpcyks10ps27v4w_vlbb6s00000gn/T/ipykernel_37869/328123281.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfOwnOutput21['매출회사id'] = dfOwnOutput21['firmid']
/var/folders/xw/1rpcyks10ps27v4w_vlbb6s00000gn/T/ipykernel_37869/328123281.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfOwnOutput21['매입회사id'] = dfOwnOutput21['firmid']
/var/folders/xw/1rpcyks10ps27v4w_vlbb6s00000gn/T/ipykernel_37869/328123281.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [577]:
dfCmpnySumry21['firmid'] = dfCmpnySumry21['firmid'].astype('int')

In [498]:
dfOwnInput21['firmid'] = dfOwnInput21['firmid'].astype('int')

In [560]:
dfOwnOutput21['firmid'] = dfOwnOutput21['firmid'].astype('int')

In [569]:
dfOwnOutput21 = dfOwnOutput21.drop_duplicates()

In [578]:
dfCmpnySumry21 = dfCmpnySumry21.merge(dfOwnInput21[dfOwnInput21['type'] == 0][['firmid', '직접지분율']], on='firmid', how='left')\
                               .merge(dfOwnOutput21[['firmid', '실질지분율']], on='firmid', how='left')
dfCmpnySumry21.head()

,publicYm,fisclYr,unityGrupNm,entrprsNm,rprsntvNm,fondDe,grinil,indutyCode,indutyNm,ordtmEmplyCo,...,caplAmount,selngAmount,thstrmNtpfAmount,reconstrctur,jurirno,year2,rcmgCode,firmid,직접지분율,실질지분율
0,202105,2020,롯데,롯데쇼핑,강희태,19700702,19870401.0,G47,소매업; 자동차 제외,24963,...,1.410000e+11,8.710000e+12,-1.030000e+12,일반회사,1100000000000,2021,620068,620068,0.120768,0.190298
1,202105,2020,지에스,지에스건설,임병용,19691219,20050404.0,F41,종합 건설업,6928,...,4.040000e+11,8.890000e+12,3.270000e+11,일반회사,1100000000000,2021,500143,500143,0.223957,0.223957
2,202105,2020,SM,삼라,조유선,19690826,20040501.0,F41,종합 건설업,12,...,1.685000e+10,1.490000e+11,7.785100e+10,일반회사,1100000000000,2021,500429,500429,0.688212,0.688212
3,202105,2020,한화,한화,옥경석 외,19521028,19870401.0,C20,화학 물질 및 화학제품 제조업; 의약품 제외,5026,...,4.900000e+11,4.000000e+12,1.290000e+11,일반회사,1100000000000,2021,350893,350893,0.267595,0.319317
4,202105,2020,동원,동원산업,이명우,19690416,20030128.0,A03,어업,761,...,1.838800e+10,7.020000e+11,7.895000e+10,일반회사,1100000000000,2021,100030,100030,0.000000,0.593151


In [501]:
dfTrans21Dmstc = dfTrans21Dmstc.merge(dfOwnInput21[dfOwnInput21['type'] == 0][['매출회사id', '직접지분율']], on='매출회사id', how='left')\
                               .merge(dfOwnOutput21[['매출회사id', '매출회사 실질지분율']], on='매출회사id', how='left')\
                               .merge(dfOwnOutput21[['매입회사id', '매입회사 실질지분율']], on='매입회사id', how='left')

,매출회사,매입회사,매출액,국내계열사계(매출액),해외계열사계(매출액),국내매출액,해외매출액,매출액총계,year2,year,...,매입사금융/비금융,grpname,grpcode,매출회사id,매입회사id,매출회사 실질지분율_x,매입회사 실질지분율_x,직접지분율,매출회사 실질지분율_y,매입회사 실질지분율_y
0,삼성전자,삼성물산,86705.0,3142513.0,141902290.0,19833135.0,146478056.0,166311191.0,2021,2020,...,비금융,samsung,511,380725,810053,0.085685,0.316255,0.051113,0.085685,0.316255
1,삼성전자,삼성물산,86705.0,3142513.0,141902290.0,19833135.0,146478056.0,166311191.0,2021,2020,...,비금융,samsung,511,380725,810053,0.085685,0.316255,0.051113,0.085685,0.316255
2,삼성전자,삼성물산,86705.0,3142513.0,141902290.0,19833135.0,146478056.0,166311191.0,2021,2020,...,비금융,samsung,511,380725,810053,0.085685,0.316255,0.051113,0.085685,0.316255
3,삼성전자,삼성물산,86705.0,3142513.0,141902290.0,19833135.0,146478056.0,166311191.0,2021,2020,...,비금융,samsung,511,380725,810053,0.085685,0.316255,0.051113,0.085685,0.316255
4,삼성전자,삼성물산,86705.0,3142513.0,141902290.0,19833135.0,146478056.0,166311191.0,2021,2020,...,비금융,samsung,511,380725,810053,0.085685,0.316255,0.051113,0.085685,0.316255


In [638]:
dfTrans21Dmstc = dfTrans21Dmstc.drop_duplicates()

In [505]:
dfTrans21Dmstc.head()

,매출회사,매입회사,매출액,국내계열사계(매출액),해외계열사계(매출액),국내매출액,해외매출액,매출액총계,year2,year,...,매출액Val,매입사상/비,매입사금융/비금융,grpname,grpcode,매출회사id,매입회사id,매출회사 실질지분율,매입회사 실질지분율,직접지분율
0,삼성전자,삼성물산,86705.0,3142513.0,141902290.0,19833135.0,146478056.0,166311191.0,2021,2020,...,1.660000e+14,상장,비금융,samsung,511,380725,810053,0.085685,0.316255,0.051113
1,삼성전자,삼성물산,86705.0,3142513.0,141902290.0,19833135.0,146478056.0,166311191.0,2021,2020,...,1.660000e+14,상장,비금융,samsung,511,380725,810053,0.085685,0.316255,0.051113
2,삼성전자,삼성물산,86705.0,3142513.0,141902290.0,19833135.0,146478056.0,166311191.0,2021,2020,...,1.660000e+14,상장,비금융,samsung,511,380725,810053,0.085685,0.316255,0.051113
3,삼성전자,삼성물산,86705.0,3142513.0,141902290.0,19833135.0,146478056.0,166311191.0,2021,2020,...,1.660000e+14,상장,비금융,samsung,511,380725,810053,0.085685,0.316255,0.051113
4,삼성전자,삼성물산,86705.0,3142513.0,141902290.0,19833135.0,146478056.0,166311191.0,2021,2020,...,1.660000e+14,상장,비금융,samsung,511,380725,810053,0.085685,0.316255,0.051113


In [502]:
dfTrans21Dmstc.columns

Index(['매출회사', '매입회사', '매출액', '국내계열사계(매출액)', '해외계열사계(매출액)', '국내매출액', '해외매출액',
       '매출액총계', 'year2', 'year', 'grpname2', '매출사상/비', '매출사금융/비금융', '매출액Val',
       '매입사상/비', '매입사금융/비금융', 'grpname', 'grpcode', '매출회사id', '매입회사id',
       '매출회사 실질지분율_x', '매입회사 실질지분율_x', '직접지분율', '매출회사 실질지분율_y',
       '매입회사 실질지분율_y'],
      dtype='object')

In [504]:
dfTrans21Dmstc = dfTrans21Dmstc.drop(columns=['매출회사 실질지분율_y', '매입회사 실질지분율_y'])
dfTrans21Dmstc.columns = ['매출회사', '매입회사', '매출액', '국내계열사계(매출액)', '해외계열사계(매출액)', '국내매출액', '해외매출액',
       '매출액총계', 'year2', 'year', 'grpname2', '매출사상/비', '매출사금융/비금융', '매출액Val',
       '매입사상/비', '매입사금융/비금융', 'grpname', 'grpcode', '매출회사id', '매입회사id',
       '매출회사 실질지분율', '매입회사 실질지분율', '직접지분율',]

In [522]:
def shareRatioType(row):
    if row['직접지분율'] < 0.2:
        return '20% 미만'
    elif row['직접지분율'] < 0.3:
        return '20~30%'
    elif row['직접지분율'] >= 0.3:
        return '30% 이상'
    else:
        return '기타'
    
def realShareRatioType(row):
    if row['매출회사 실질지분율'] < 0.2:
        return '20% 미만'
    elif row['매출회사 실질지분율'] < 0.3:
        return '20~30%'
    elif row['매출회사 실질지분율'] >= 0.3:
        return '30% 이상'
    else:
        return '기타'
    
def realShareRatioType2(row):
    if row['실질지분율'] < 0.2:
        return '20% 미만'
    elif row['실질지분율'] < 0.3:
        return '20~30%'
    elif row['실질지분율'] >= 0.3:
        return '30% 이상'
    else:
        return '기타'    

def realShareRatioDiffType(row):
    if row['매출회사 실질지분율'] - row['매입회사 실질지분율']  < 0:
        return '0%p 미만'
    elif row['매출회사 실질지분율'] - row['매입회사 실질지분율'] < 0.03:
        return '0~3%p'
    elif row['매출회사 실질지분율'] - row['매입회사 실질지분율'] >= 0.03:
        return '3%p 이상'
    else:
        return '기타'

In [520]:
dfTrans18Dmstc['직접지분율 유형'] = dfTrans18Dmstc.apply(shareRatioType, axis=1)
dfTrans18Dmstc['실질지분율 유형'] = dfTrans18Dmstc.apply(realShareRatioType, axis=1)
dfTrans18Dmstc['실질지분율 차이 유형'] = dfTrans18Dmstc.apply(realShareRatioDiffType, axis=1)
dfTrans18Dmstc.head()

,year,year2,grpname,grpname2,grpcode,매출회사,매출회사id,매출사\n상/비,매입회사,매입회사id,...,매출회사\n해외매출액,재벌순위,매출사 CFR\n(우선주포함),매출사 FAM\n(우선주포함),직접지분율,매출회사 실질지분율,매입회사 실질지분율,직접지분율 유형,실질지분율 유형,실질지분율 차이 유형
0,2017,2018,DB,DB,365,DBInc.,352098,상장,DB금융투자,821640,...,76213.0,36,0.401239,0.401239,0.401239,0.401239,0.174456,30% 이상,30% 이상,3%p 이상
1,2017,2018,DB,DB,365,DBInc.,352098,상장,DB라이텍,137656,...,76213.0,36,0.401239,0.401239,0.401239,0.401239,0.060650,30% 이상,30% 이상,3%p 이상
2,2017,2018,DB,DB,365,DBInc.,352098,상장,DB메탈,12097776,...,76213.0,36,0.401239,0.401239,0.401239,0.401239,0.589516,30% 이상,30% 이상,0%p 미만
3,2017,2018,DB,DB,365,DBInc.,352098,상장,DB생명보험,821420,...,76213.0,36,0.401239,0.401239,0.401239,0.401239,0.141807,30% 이상,30% 이상,3%p 이상
4,2017,2018,DB,DB,365,DBInc.,352098,상장,DB손해보험,870099,...,76213.0,36,0.401239,0.401239,0.401239,0.401239,0.181040,30% 이상,30% 이상,3%p 이상


In [523]:
dfCmpnySumry18C['직접지분율 유형'] = dfCmpnySumry18C.apply(shareRatioType, axis=1)
dfCmpnySumry18C['실질지분율 유형'] = dfCmpnySumry18C.apply(realShareRatioType2, axis=1)

In [526]:
dfCmpnySumry18C.head()

,publicYm,fisclYr,unityGrupNm,entrprsNm,rprsntvNm,fondDe,grinil,indutyCode,indutyNm,ordtmEmplyCo,...,thstrmNtpfAmount,reconstrctur,jurirno,year2,rcmgCode,firmid,직접지분율,실질지분율,직접지분율 유형,실질지분율 유형
0,201805,2017,DB,(주)동부월드,최경진,1989/05/15,1997/06/02,R91,스포츠 및 오락관련 서비스업,76,...,"-3,672",회생절차,1101110628086,2018,910813.0,910813.0,0.000000,0.099492,20% 미만,20% 미만
1,201805,2017,DB,디비씨앤에스자동차손해사정(주),문수원,2011/03/17,2011/04/05,K66,금융 및 보험 관련 서비스업,742,...,"1,388",금융보험업,1101114558669,2018,18004172.0,18004172.0,0.000000,0.181040,20% 미만,20% 미만
2,201805,2017,DB,디비캐피탈(주),윤재인,1995/12/08,1995/12/08,K64,금융업,20,...,"2,344",금융보험업,1101111220641,2018,56018.0,56018.0,0.000000,0.166868,20% 미만,20% 미만
3,201805,2017,DB,디비금융투자(주),고원종,1982/12/20,1982/12/20,K66,금융 및 보험 관련 서비스업,968,...,"2,591",금융보험업,1101110339956,2018,821640.0,821640.0,0.129058,0.174456,20% 미만,20% 미만
4,201805,2017,DB,디비생명보험(주),이태운,1989/04/14,1997/01/03,K65,보험 및 연금업,569,...,"29,215",금융보험업,1101110623606,2018,821420.0,821420.0,0.001217,0.141807,20% 미만,20% 미만


In [543]:
dfTrans19Dmstc['직접지분율 유형'] = dfTrans19Dmstc.apply(shareRatioType, axis=1)
dfTrans19Dmstc['실질지분율 유형'] = dfTrans19Dmstc.apply(realShareRatioType, axis=1)
dfTrans19Dmstc['실질지분율 차이 유형'] = dfTrans19Dmstc.apply(realShareRatioDiffType, axis=1)
dfCmpnySumry19['직접지분율 유형'] = dfCmpnySumry19.apply(shareRatioType, axis=1)
dfCmpnySumry19['실질지분율 유형'] = dfCmpnySumry19.apply(realShareRatioType2, axis=1)

In [545]:
dfTrans20Dmstc['직접지분율 유형'] = dfTrans20Dmstc.apply(shareRatioType, axis=1)
dfTrans20Dmstc['실질지분율 유형'] = dfTrans20Dmstc.apply(realShareRatioType, axis=1)
dfTrans20Dmstc['실질지분율 차이 유형'] = dfTrans20Dmstc.apply(realShareRatioDiffType, axis=1)
dfCmpnySumry20['직접지분율 유형'] = dfCmpnySumry20.apply(shareRatioType, axis=1)
dfCmpnySumry20['실질지분율 유형'] = dfCmpnySumry20.apply(realShareRatioType2, axis=1)

In [580]:
dfTrans21Dmstc['직접지분율 유형'] = dfTrans21Dmstc.apply(shareRatioType, axis=1)
dfTrans21Dmstc['실질지분율 유형'] = dfTrans21Dmstc.apply(realShareRatioType, axis=1)
dfTrans21Dmstc['실질지분율 차이 유형'] = dfTrans21Dmstc.apply(realShareRatioDiffType, axis=1)
dfCmpnySumry21['직접지분율 유형'] = dfCmpnySumry21.apply(shareRatioType, axis=1)
dfCmpnySumry21['실질지분율 유형'] = dfCmpnySumry21.apply(realShareRatioType2, axis=1)

In [ ]:
dfCmpnySumry18C.groupby('직접지분율 유형')['entrprsNm'].count()

In [547]:
dfCmpnySumry19.groupby('직접지분율 유형')['entrprsNm'].count()

직접지분율 유형
20% 미만    1549
20~30%      39
30% 이상     187
기타         436
Name: entrprsNm, dtype: int64

In [548]:
dfCmpnySumry20.groupby('직접지분율 유형')['entrprsNm'].count()

직접지분율 유형
20% 미만    1377
20~30%      40
30% 이상     194
기타         525
Name: entrprsNm, dtype: int64

In [582]:
dfCmpnySumry21.groupby('직접지분율 유형')['entrprsNm'].count()

직접지분율 유형
20% 미만    1752
20~30%      43
30% 이상     247
기타         417
Name: entrprsNm, dtype: int64

In [584]:
dfCmpnySumry18C.groupby('직접지분율 유형')['entrprsNm'].count() / dfCmpnySumry18C[dfCmpnySumry18C['직접지분율 유형'] != '기타']['entrprsNm'].count()

직접지분율 유형
20% 미만    0.886156
20~30%    0.021739
30% 이상    0.092105
기타        0.204233
Name: entrprsNm, dtype: float64

In [585]:
dfCmpnySumry19.groupby('직접지분율 유형')['entrprsNm'].count() / dfCmpnySumry19[dfCmpnySumry19['직접지분율 유형'] != '기타']['entrprsNm'].count()

직접지분율 유형
20% 미만    0.872676
20~30%    0.021972
30% 이상    0.105352
기타        0.245634
Name: entrprsNm, dtype: float64

In [586]:
dfCmpnySumry20.groupby('직접지분율 유형')['entrprsNm'].count() / dfCmpnySumry20[dfCmpnySumry20['직접지분율 유형'] != '기타']['entrprsNm'].count()

직접지분율 유형
20% 미만    0.854749
20~30%    0.024829
30% 이상    0.120422
기타        0.325885
Name: entrprsNm, dtype: float64

In [639]:
dfCmpnySumry21.groupby('직접지분율 유형')['entrprsNm'].count() / dfCmpnySumry21[dfCmpnySumry21['직접지분율 유형'] != '기타']['entrprsNm'].count()

직접지분율 유형
20% 미만    0.857982
20~30%    0.021058
30% 이상    0.120960
기타        0.204212
Name: entrprsNm, dtype: float64

In [533]:
dfTrans18Dmstc.groupby('직접지분율 유형').size() / dfCmpnySumry18C.groupby('직접지분율 유형')['entrprsNm'].count()

직접지분율 유형
20% 미만     5.509361
20~30%    10.500000
30% 이상     7.428571
기타         0.759104
dtype: float64

In [588]:
dfTrans19Dmstc.groupby('직접지분율 유형').size() / dfCmpnySumry19.groupby('직접지분율 유형')['entrprsNm'].count()

직접지분율 유형
20% 미만     6.540349
20~30%    14.794872
30% 이상     5.155080
기타         3.676606
dtype: float64

In [589]:
dfTrans20Dmstc.groupby('직접지분율 유형').size() / dfCmpnySumry20.groupby('직접지분율 유형')['entrprsNm'].count()

직접지분율 유형
20% 미만     5.881627
20~30%    12.575000
30% 이상     4.654639
기타         3.552381
dtype: float64

In [640]:
dfTrans21Dmstc.groupby('직접지분율 유형').size() / dfCmpnySumry21.groupby('직접지분율 유형')['entrprsNm'].count()

직접지분율 유형
20% 미만     5.206621
20~30%    12.162791
30% 이상     4.194332
기타         2.340528
dtype: float64

In [534]:
dfTrans18Dmstc.groupby('직접지분율 유형')['매출액'].sum() / dfCmpnySumry18C.groupby('직접지분율 유형')['entrprsNm'].count()

직접지분율 유형
20% 미만     97059.001762
20~30%    202293.657895
30% 이상     79715.236025
기타          5099.221289
dtype: float64

In [591]:
dfTrans19Dmstc.groupby('직접지분율 유형')['매출액'].sum() / dfCmpnySumry19.groupby('직접지분율 유형')['entrprsNm'].count()

직접지분율 유형
20% 미만    115856.075965
20~30%    274939.846154
30% 이상     47509.354545
기타         17754.692661
dtype: float64

In [592]:
dfTrans20Dmstc.groupby('직접지분율 유형')['매출액'].sum() / dfCmpnySumry20.groupby('직접지분율 유형')['entrprsNm'].count()

직접지분율 유형
20% 미만     97726.136616
20~30%    255704.000000
30% 이상     45008.938144
기타         46918.690857
dtype: float64

In [641]:
dfTrans21Dmstc.groupby('직접지분율 유형')['매출액'].sum() / dfCmpnySumry21.groupby('직접지분율 유형')['entrprsNm'].count()

직접지분율 유형
20% 미만     79330.197775
20~30%    226041.674419
30% 이상     37545.522267
기타         10149.717506
dtype: float64

In [598]:
dfCmpnySumry18C['selngAmount'] = dfCmpnySumry18C['selngAmount'].replace(',', '', regex=True).astype(float)
dfCmpnySumry19['selngAmount'] = dfCmpnySumry19['selngAmount'].replace(',', '', regex=True).astype(float)
dfCmpnySumry20['selngAmount'] = dfCmpnySumry20['selngAmount'].replace(',', '', regex=True).astype(float)
dfCmpnySumry21['selngAmount'] = dfCmpnySumry21['selngAmount'].replace(',', '', regex=True).astype(float)

In [541]:
dfTrans18Dmstc.groupby('직접지분율 유형')['매출액'].sum() / dfCmpnySumry18C.groupby('직접지분율 유형')['selngAmount'].sum()

직접지분율 유형
20% 미만    0.127683
20~30%    0.080127
30% 이상    0.148581
기타        0.023277
dtype: float64

In [600]:
dfTrans19Dmstc.groupby('직접지분율 유형')['매출액'].sum() / dfCmpnySumry19.groupby('직접지분율 유형')['selngAmount'].sum()

직접지분율 유형
20% 미만    0.150244
20~30%    0.104888
30% 이상    0.117371
기타        0.084351
dtype: float64

In [601]:
dfTrans20Dmstc.groupby('직접지분율 유형')['매출액'].sum() / dfCmpnySumry20.groupby('직접지분율 유형')['selngAmount'].sum()

직접지분율 유형
20% 미만    0.131109
20~30%    0.099993
30% 이상    0.118681
기타        0.118596
dtype: float64

In [642]:
dfTrans21Dmstc.groupby('직접지분율 유형')['매출액'].sum() / dfCmpnySumry21.groupby('직접지분율 유형')['selngAmount'].sum() * 100000

직접지분율 유형
20% 미만    0.012097
20~30%    0.008201
30% 이상    0.012520
기타        0.008381
dtype: float64

# 실질지분율

In [606]:
dfCmpnySumry18C.groupby('실질지분율 유형')['entrprsNm'].count()

실질지분율 유형
20% 미만    996
20~30%    231
30% 이상    587
기타        291
Name: entrprsNm, dtype: int64

In [607]:
dfCmpnySumry19.groupby('실질지분율 유형')['entrprsNm'].count()

실질지분율 유형
20% 미만    1170
20~30%     287
30% 이상     643
기타         111
Name: entrprsNm, dtype: int64

In [608]:
dfCmpnySumry20.groupby('실질지분율 유형')['entrprsNm'].count()

실질지분율 유형
20% 미만    1144
20~30%     294
30% 이상     676
기타          22
Name: entrprsNm, dtype: int64

In [643]:
dfCmpnySumry21.groupby('실질지분율 유형')['entrprsNm'].count()

실질지분율 유형
20% 미만    1262
20~30%     331
30% 이상     834
기타          32
Name: entrprsNm, dtype: int64

In [644]:
dfCmpnySumry18C.groupby('실질지분율 유형')['entrprsNm'].count() / dfCmpnySumry18C[dfCmpnySumry18C['실질지분율 유형'] != '기타']['entrprsNm'].count()

실질지분율 유형
20% 미만    0.549063
20~30%    0.127343
30% 이상    0.323594
기타        0.160419
Name: entrprsNm, dtype: float64

In [611]:
dfCmpnySumry19.groupby('실질지분율 유형')['entrprsNm'].count() / dfCmpnySumry19[dfCmpnySumry19['실질지분율 유형'] != '기타']['entrprsNm'].count()

실질지분율 유형
20% 미만    0.557143
20~30%    0.136667
30% 이상    0.306190
기타        0.052857
Name: entrprsNm, dtype: float64

In [612]:
dfCmpnySumry20.groupby('실질지분율 유형')['entrprsNm'].count() / dfCmpnySumry20[dfCmpnySumry20['실질지분율 유형'] != '기타']['entrprsNm'].count()

실질지분율 유형
20% 미만    0.541154
20~30%    0.139073
30% 이상    0.319773
기타        0.010407
Name: entrprsNm, dtype: float64

In [645]:
dfCmpnySumry21.groupby('실질지분율 유형')['entrprsNm'].count() / dfCmpnySumry21[dfCmpnySumry21['실질지분율 유형'] != '기타']['entrprsNm'].count()

실질지분율 유형
20% 미만    0.519984
20~30%    0.136382
30% 이상    0.343634
기타        0.013185
Name: entrprsNm, dtype: float64

In [614]:
dfTrans18Dmstc.groupby('실질지분율 유형').size() / dfCmpnySumry18C.groupby('실질지분율 유형')['entrprsNm'].count()

실질지분율 유형
20% 미만    5.953815
20~30%    6.142857
30% 이상    5.178876
기타        0.037801
dtype: float64

In [615]:
dfTrans19Dmstc.groupby('실질지분율 유형').size() / dfCmpnySumry19.groupby('실질지분율 유형')['entrprsNm'].count()

실질지분율 유형
20% 미만    6.624786
20~30%    7.480836
30% 이상    4.726283
기타        3.045045
dtype: float64

In [616]:
dfTrans20Dmstc.groupby('실질지분율 유형').size() / dfCmpnySumry20.groupby('실질지분율 유형')['entrprsNm'].count()

실질지분율 유형
20% 미만    5.785839
20~30%    6.136054
30% 이상    4.352071
기타        0.227273
dtype: float64

In [646]:
dfTrans21Dmstc.groupby('실질지분율 유형').size() / dfCmpnySumry21.groupby('실질지분율 유형')['entrprsNm'].count()

실질지분율 유형
20% 미만    5.012678
20~30%    6.072508
30% 이상    3.928058
기타        1.406250
dtype: float64

In [618]:
dfTrans18Dmstc.groupby('실질지분율 유형')['매출액'].sum() / dfCmpnySumry18C.groupby('실질지분율 유형')['entrprsNm'].count()

실질지분율 유형
20% 미만    120440.845843
20~30%    112063.311818
30% 이상     45693.501261
기타            63.006873
dtype: float64

In [619]:
dfTrans19Dmstc.groupby('실질지분율 유형')['매출액'].sum() / dfCmpnySumry19.groupby('실질지분율 유형')['entrprsNm'].count()

실질지분율 유형
20% 미만    125318.573803
20~30%    118635.187526
30% 이상     36348.519129
기타         24917.864865
dtype: float64

In [620]:
dfTrans20Dmstc.groupby('실질지분율 유형')['매출액'].sum() / dfCmpnySumry20.groupby('실질지분율 유형')['entrprsNm'].count()

실질지분율 유형
20% 미만    114155.368418
20~30%     88785.611735
30% 이상     31575.111686
기타          5436.818182
dtype: float64

In [647]:
dfTrans21Dmstc.groupby('실질지분율 유형')['매출액'].sum() / dfCmpnySumry21.groupby('실질지분율 유형')['entrprsNm'].count()

실질지분율 유형
20% 미만    93129.256958
20~30%    72289.758066
30% 이상    24813.979017
기타         1893.250000
dtype: float64

In [622]:
dfTrans18Dmstc.groupby('실질지분율 유형')['매출액'].sum() / dfCmpnySumry18C.groupby('실질지분율 유형')['selngAmount'].sum()

실질지분율 유형
20% 미만    0.118393
20~30%    0.123566
30% 이상    0.157187
기타        0.000411
dtype: float64

In [623]:
dfTrans19Dmstc.groupby('실질지분율 유형')['매출액'].sum() / dfCmpnySumry19.groupby('실질지분율 유형')['selngAmount'].sum()

실질지분율 유형
20% 미만    0.141935
20~30%    0.142190
30% 이상    0.138307
기타        0.121914
dtype: float64

In [624]:
dfTrans20Dmstc.groupby('실질지분율 유형')['매출액'].sum() / dfCmpnySumry20.groupby('실질지분율 유형')['selngAmount'].sum()

실질지분율 유형
20% 미만    0.128252
20~30%    0.118872
30% 이상    0.124575
기타        0.156819
dtype: float64

In [648]:
dfTrans21Dmstc.groupby('실질지분율 유형')['매출액'].sum() / dfCmpnySumry21.groupby('실질지분율 유형')['selngAmount'].sum() * 100000

실질지분율 유형
20% 미만    0.011577
20~30%    0.012077
30% 이상    0.011585
기타        0.107644
dtype: float64

# 실질지분율 차이

In [637]:
dfTrans18Dmstc.to_excel('../data/transactions-report/transactions-report2018.xlsx', index=False)
dfTrans19Dmstc.to_excel('../data/transactions-report/transactions-report2019.xlsx', index=False)
dfTrans20Dmstc.to_excel('../data/transactions-report/transactions-report2020.xlsx', index=False)
dfTrans21Dmstc.to_excel('../data/transactions-report/transactions-report2021.xlsx', index=False)

In [649]:
dfCmpnySumry18C.to_excel("../data/transactions-report/cmpnySurmry2018.xlsx")
dfCmpnySumry19.to_excel("../data/transactions-report/cmpnySurmry2019.xlsx")
dfCmpnySumry20.to_excel("../data/transactions-report/cmpnySurmry2020.xlsx")
dfCmpnySumry21.to_excel("../data/transactions-report/cmpnySurmry2021.xlsx")

In [ ]:
dfOwnInput18.to_excel("../data/almeida/2018/in/almeida_input_2018.xlsx")
dfOwnInput19.to_excel("../data/almeida/2019/in/almeida_input_2019.xlsx")
dfOwnInput20.to_excel("../data/almeida/2020/in/almeida_input_2020.xlsx")
dfOwnInput21.to_excel("../data/almeida/2021/in/almeida_input_2021.xlsx")